In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import boto3
import io
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
from scipy.stats import chi2_contingency, fisher_exact
from statsmodels.stats.proportion import proportions_ztest
from sklearn.linear_model import LogisticRegression
from scipy.stats import chi2_contingency

# Import required libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
import itertools
import warnings

## S3

In [2]:
# ───────────── Settings ─────────────
BUCKET       = 'bdc-public-curated'
PREFIX       = 'ndacan/nytd/outcomes/waves_processed/'

# ───────────── Initialize S3 client & list files ─────────────
s3 = boto3.client('s3')

In [3]:
# Load AFCARS_TN_processed(in).csv as a DataFrame
afcars_tn_df = pd.read_csv('AFCARS_TN_processed(in).csv', dtype=str)


## Helpers

In [4]:
def s3_read_csv(key):
    """Load a CSV from S3 into a pandas DataFrame."""
    obj = s3.get_object(Bucket=BUCKET, Key=key)
    return pd.read_csv(io.BytesIO(obj['Body'].read()), dtype=str)

In [5]:
# ID and metadata columns
ID_COLS   = ['StFIPS', 'RepDate',]


In [6]:
def normalize_keys(df, wave_tag):
    # 1) rename StFIPS_wX → StFIPS, etc.
    for col in ID_COLS:
        suff = f"{col}_{wave_tag}"
        if suff in df.columns:
            df.rename(columns={suff: col}, inplace=True)
    # 2) trim whitespace & unify types
    df[ID_COLS] = df[ID_COLS].astype(str).apply(lambda s: s.str.strip())
    # 3) parse dates
    df['RepDate'] = pd.to_datetime(df['RepDate'], errors='coerce')
    return df

In [7]:
# ───────────── 1) LOAD & NORMALIZE WAVES ─────────────
wave1 = s3_read_csv(f"{PREFIX}wave1.csv")
wave2 = s3_read_csv(f"{PREFIX}wave2.csv")
wave3 = s3_read_csv(f"{PREFIX}wave3.csv")
all_wave = s3_read_csv(f"{PREFIX}cleaned_all_waves.csv")
wave1 = normalize_keys(wave1, "w1")
wave2 = normalize_keys(wave2, "w2")
wave3 = normalize_keys(wave3, "w3")

print("Wave shapes:", wave1.shape, wave2.shape, wave3.shape)


Wave shapes: (3068, 38) (1362, 38) (950, 38)


## Wave 1

In [8]:
# 2) Drop any column suffixed with '_w2' or '_w3'
wave1 = all_wave.drop(
    columns=[c for c in all_wave.columns if c.endswith(('_w23'))]
).copy()

# 3) Optionally strip off the '_w1' suffix so your columns are the “base” names
wave1.columns = [
    re.sub(r'_w1$', '', c)
    for c in wave1.columns
]

In [9]:
# print the (rows, columns) tuple
print(wave1.shape)

(720, 42)


## Wave 2-3

In [10]:
# 2) Drop any column suffixed with '_w2' or '_w3'
wave23 = all_wave.drop(
    columns=[c for c in all_wave.columns if c.endswith(('_w1'))]
).copy()

# 3) Optionally strip off the '_w1' suffix so your columns are the “base” names
wave23.columns = [
    re.sub(r'_w1$', '', c)
    for c in wave23.columns
]

In [11]:
# print the (rows, columns) tuple
print(wave23.shape)

(720, 41)


In [12]:
# Cell 2: Define predictors and outcomes
OUTCOMES = [
    'Homeless', 'SubAbuse', 'Incarc',
    'Children', 'Marriage', 'CnctAdult'
]

PREDICTORS = [
    'CurrFTE', 'CurrPTE', 'EmplySklls', 'SocSecrty', 'EducAid',
    'PubFinAs', 'PubFoodAs', 'PubHousAs', 'OthrFinAs',
    'Medicaid', 'OthrHlthIn', 'MedicalIn', 'MentlHlthIn'
]

print(f"Predictors: {PREDICTORS}")
print(f"Outcomes: {OUTCOMES}")

Predictors: ['CurrFTE', 'CurrPTE', 'EmplySklls', 'SocSecrty', 'EducAid', 'PubFinAs', 'PubFoodAs', 'PubHousAs', 'OthrFinAs', 'Medicaid', 'OthrHlthIn', 'MedicalIn', 'MentlHlthIn']
Outcomes: ['Homeless', 'SubAbuse', 'Incarc', 'Children', 'Marriage', 'CnctAdult']


# Stats

In [13]:
# Cell 6: Create longitudinal dataset and overall analysis - FIXED
def create_longitudinal_data(wave1_df, wave23_df):
    """Merge wave1 predictors with wave23 outcomes by StFCID"""
    
    # Get predictors from wave1 (clean names)
    wave1_predictors_df = wave1_df[['StFCID'] + [col for col in PREDICTORS if col in wave1_df.columns]].copy()
    
    # Get outcomes from wave23 (with suffixes)
    wave23_outcome_cols = ['StFCID']
    for outcome in OUTCOMES:
        # Add both _w2 and _w3 versions if they exist
        wave23_outcome_cols.extend([col for col in wave23_df.columns if col.startswith(outcome + '_w')])
    
    wave23_outcomes_df = wave23_df[wave23_outcome_cols].copy()
    
    # Merge on StFCID
    longitudinal_df = pd.merge(wave1_predictors_df, wave23_outcomes_df, on='StFCID', how='inner')
    
    print(f"Wave1 predictors shape: {wave1_predictors_df.shape}")
    print(f"Wave23 outcomes shape: {wave23_outcomes_df.shape}")
    print(f"Wave23 outcome columns found: {[col for col in wave23_outcome_cols if col != 'StFCID']}")
    print(f"Longitudinal merged shape: {longitudinal_df.shape}")
    
    return longitudinal_df

# Create longitudinal dataset
longitudinal_data = create_longitudinal_data(wave1, wave23)

Wave1 predictors shape: (720, 14)
Wave23 outcomes shape: (720, 7)
Wave23 outcome columns found: ['Homeless_w23', 'SubAbuse_w23', 'Incarc_w23', 'Children_w23', 'Marriage_w23', 'CnctAdult_w23']
Longitudinal merged shape: (720, 20)


# Whole Dataframe

In [15]:
# Cell 1: Check StFCID columns and basic info
print("Checking StFCID column in both datasets...")
print("="*50)

print(f"Wave1 shape: {wave1.shape}")
print(f"Wave23 shape: {wave23.shape}")

# Check if StFCID exists
print(f"\nStFCID in wave1: {'StFCID' in wave1.columns}")
print(f"StFCID in wave23: {'StFCID' in wave23.columns}")

# Show unique StFCID counts
wave1_ids = wave1['StFCID'].dropna().unique()
wave23_ids = wave23['StFCID'].dropna().unique()

print(f"\nUnique StFCID values:")
print(f"  Wave1: {len(wave1_ids):,} unique IDs")
print(f"  Wave23: {len(wave23_ids):,} unique IDs")

# Find matching IDs
matching_ids = set(wave1_ids).intersection(set(wave23_ids))
print(f"  Matching between both waves: {len(matching_ids):,} IDs")

# Sample of matching IDs
print(f"\nSample matching StFCID values:")
print(list(matching_ids)[:10])


Checking StFCID column in both datasets...
Wave1 shape: (720, 42)
Wave23 shape: (720, 41)

StFCID in wave1: True
StFCID in wave23: True

Unique StFCID values:
  Wave1: 720 unique IDs
  Wave23: 720 unique IDs
  Matching between both waves: 720 IDs

Sample matching StFCID values:
['TNOOOONMNKJMJM', 'TNOOOOOKLONFKG', 'TNOOOOOHIJKKNM', 'TNOOOONOGGFHJK', 'TNµ®œ¿Š¤\xad©þ±ü£', 'TNOOOOMGNKLFGG', 'TNOOOOLJOHNFML', 'TNOOOOMKLMKFJN', 'TNOOOOOKMGNOKI', 'TNOOOOMKGKFLKL']


In [16]:
# Cell 2: Create combo dataset with matching StFCIDs
print("\nCreating combo dataset with matching StFCIDs...")
print("="*50)

# Simple inner join on StFCID - only keep records that exist in both datasets
combo_dataset = pd.merge(
    wave1,
    wave23,
    on='StFCID',
    how='inner',
    suffixes=('_w1', '_w23')
)

print(f"Combo dataset created!")
print(f"  Original Wave1: {len(wave1):,} records")
print(f"  Original Wave23: {len(wave23):,} records") 
print(f"  Combo dataset: {len(combo_dataset):,} records")
print(f"  Retention rate: {len(combo_dataset)/min(len(wave1), len(wave23))*100:.1f}%")

print(f"\nCombo dataset shape: {combo_dataset.shape}")
print(f"Columns: {len(combo_dataset.columns)} total")


Creating combo dataset with matching StFCIDs...
Combo dataset created!
  Original Wave1: 720 records
  Original Wave23: 720 records
  Combo dataset: 720 records
  Retention rate: 100.0%

Combo dataset shape: (720, 82)
Columns: 82 total


In [17]:
 # write the first 5 rows (head) to CSV, without the DataFrame index
combo_dataset.to_csv('combo_dataset.csv')

In [18]:
# Cell 3: Analyze the combo dataset structure - FIXED 
print("\nAnalyzing combo dataset structure...")
print("="*50)

# Count columns by source
actual_w1_cols = [col for col in combo_dataset.columns if col.endswith('_w1')]
no_suffix_cols = [col for col in combo_dataset.columns if not col.endswith(('_w1', '_w23'))]
w23_cols = [col for col in combo_dataset.columns if col.endswith('_w23')]

# Wave1 columns are the no_suffix_cols (these ARE the w1 data)
w1_cols = [col for col in no_suffix_cols if col != 'StFCID']  # This is the fix!
shared_cols = ['StFCID'] if 'StFCID' in combo_dataset.columns else []

print(f"Column breakdown:")
print(f"  Wave1 columns (no suffix): {len(w1_cols)}")
print(f"  Actual _w1 suffixed columns: {len(actual_w1_cols)}")
print(f"  Wave23 columns (_w23): {len(w23_cols)}")
print(f"  Shared/ID columns: {len(shared_cols)}")

print(f"\nShared columns: {shared_cols}")
print(f"\nSample Wave1 columns (no suffix): {w1_cols[:10]}")
print(f"Sample Wave23 columns: {w23_cols[:10]}")


Analyzing combo dataset structure...
Column breakdown:
  Wave1 columns (no suffix): 37
  Actual _w1 suffixed columns: 4
  Wave23 columns (_w23): 40
  Shared/ID columns: 1

Shared columns: ['StFCID']

Sample Wave1 columns (no suffix): ['RepDate', 'OutcmRpt', 'OutcmDte', 'OutcmFCS', 'CurrFTE', 'CurrPTE', 'EmplySklls', 'SocSecrty', 'EducAid', 'PubFinAs']
Sample Wave23 columns: ['StFIPS_w23', 'St_w23', 'Sex_w23', 'Cohort_w23', 'Elig19_w23', 'PubFoodAs_w23', 'SubAbuse_w23', 'Assoc_Degree_w23', 'Responded_w23', 'HS_or_GED_w23']


In [19]:
# Cell 4: Check for our specific variables - FIXED to look for exact predictor names
print("\nChecking for predictor and outcome variables...")
print("="*50)

# Check predictors - look for exact names (like 'CurrFTE') 
print("PREDICTORS in combo dataset:")
found_predictors = []
for pred in PREDICTORS:
    # For predictors: look for exact name in w1_cols (no suffix columns)
    exact_match = pred in w1_cols
    
    if exact_match:
        found_predictors.append(pred)
        print(f"  ✓ {pred}: found in Wave1 (no suffix)")
    else:
        print(f"  ✗ {pred}: not found")

print(f"\nFound {len(found_predictors)}/{len(PREDICTORS)} predictors")

# Check outcomes in Wave23 columns
print("\nOUTCOMES in combo dataset:")
found_outcomes = []
for outcome in OUTCOMES:
    # Check in w23 columns first, then no suffix
    w23_match = f"{outcome}_w23" in w23_cols
    no_suffix_match = outcome in w1_cols
    
    if w23_match or no_suffix_match:
        found_outcomes.append(outcome)
        match_info = []
        if no_suffix_match: match_info.append("Wave1")
        if w23_match: match_info.append("Wave23")
        print(f"  ✓ {outcome}: {', '.join(match_info)}")
    else:
        print(f"  ✗ {outcome}: not found")

print(f"\nFound {len(found_outcomes)}/{len(OUTCOMES)} outcomes")

# Create final column lists
predictor_columns = [pred for pred in PREDICTORS if pred in w1_cols]
outcome_columns = [f"{outcome}_w23" for outcome in OUTCOMES if f"{outcome}_w23" in w23_cols]

print(f"\nFINAL COLUMN MAPPING:")
print(f"  Predictor columns (Wave1): {predictor_columns}")
print(f"  Outcome columns (Wave23): {outcome_columns}")


Checking for predictor and outcome variables...
PREDICTORS in combo dataset:
  ✓ CurrFTE: found in Wave1 (no suffix)
  ✓ CurrPTE: found in Wave1 (no suffix)
  ✓ EmplySklls: found in Wave1 (no suffix)
  ✓ SocSecrty: found in Wave1 (no suffix)
  ✓ EducAid: found in Wave1 (no suffix)
  ✓ PubFinAs: found in Wave1 (no suffix)
  ✓ PubFoodAs: found in Wave1 (no suffix)
  ✓ PubHousAs: found in Wave1 (no suffix)
  ✓ OthrFinAs: found in Wave1 (no suffix)
  ✓ Medicaid: found in Wave1 (no suffix)
  ✓ OthrHlthIn: found in Wave1 (no suffix)
  ✓ MedicalIn: found in Wave1 (no suffix)
  ✓ MentlHlthIn: found in Wave1 (no suffix)

Found 13/13 predictors

OUTCOMES in combo dataset:
  ✓ Homeless: Wave1, Wave23
  ✓ SubAbuse: Wave1, Wave23
  ✓ Incarc: Wave1, Wave23
  ✓ Children: Wave1, Wave23
  ✓ Marriage: Wave1, Wave23
  ✓ CnctAdult: Wave1, Wave23

Found 6/6 outcomes

FINAL COLUMN MAPPING:
  Predictor columns (Wave1): ['CurrFTE', 'CurrPTE', 'EmplySklls', 'SocSecrty', 'EducAid', 'PubFinAs', 'PubFoodAs', 'Pu

In [20]:

# Cell 4: Check for our specific variables
print("\nChecking for predictor and outcome variables...")
print("="*50)

# Check predictors in Wave1 columns
print("PREDICTORS in combo dataset:")
found_predictors = []
for pred in PREDICTORS:
    # Check if predictor exists in any form
    exact_match = pred in combo_dataset.columns
    w1_match = f"{pred}_w1" in combo_dataset.columns
    w23_match = f"{pred}_w23" in combo_dataset.columns
    
    if exact_match or w1_match or w23_match:
        found_predictors.append(pred)
        match_info = []
        if exact_match: match_info.append("exact")
        if w1_match: match_info.append("w1")
        if w23_match: match_info.append("w23")
        print(f"  ✓ {pred}: {', '.join(match_info)}")
    else:
        print(f"  ✗ {pred}: not found")

print(f"\nFound {len(found_predictors)}/{len(PREDICTORS)} predictors")

# Check outcomes in Wave23 columns
print("\nOUTCOMES in combo dataset:")
found_outcomes = []
for outcome in OUTCOMES:
    # Check if outcome exists in any form
    exact_match = outcome in combo_dataset.columns
    w1_match = f"{outcome}_w1" in combo_dataset.columns
    w23_match = f"{outcome}_w23" in combo_dataset.columns
    
    if exact_match or w1_match or w23_match:
        found_outcomes.append(outcome)
        match_info = []
        if exact_match: match_info.append("exact")
        if w1_match: match_info.append("w1")
        if w23_match: match_info.append("w23")
        print(f"  ✓ {outcome}: {', '.join(match_info)}")
    else:
        print(f"  ✗ {outcome}: not found")

print(f"\nFound {len(found_outcomes)}/{len(OUTCOMES)} outcomes")



Checking for predictor and outcome variables...
PREDICTORS in combo dataset:
  ✓ CurrFTE: exact, w23
  ✓ CurrPTE: exact, w23
  ✓ EmplySklls: exact, w23
  ✓ SocSecrty: exact, w23
  ✓ EducAid: exact, w23
  ✓ PubFinAs: exact, w23
  ✓ PubFoodAs: exact, w23
  ✓ PubHousAs: exact, w23
  ✓ OthrFinAs: exact, w23
  ✓ Medicaid: exact, w23
  ✓ OthrHlthIn: exact, w23
  ✓ MedicalIn: exact, w23
  ✓ MentlHlthIn: exact, w23

Found 13/13 predictors

OUTCOMES in combo dataset:
  ✓ Homeless: exact, w23
  ✓ SubAbuse: exact, w23
  ✓ Incarc: exact, w23
  ✓ Children: exact, w23
  ✓ Marriage: exact, w23
  ✓ CnctAdult: exact, w23

Found 6/6 outcomes


In [21]:
# Cell 5: Create analysis-ready dataset
print("\nCreating analysis-ready dataset...")
print("="*50)

# Build predictor column list (use exact names from Wave1 - no suffix)
predictor_columns = []
for pred in PREDICTORS:
    if pred in w1_cols:  # Look for exact name like 'CurrFTE'
        predictor_columns.append(pred)

# Build outcome column list (prefer Wave23 _w23 versions)
outcome_columns = []
for outcome in OUTCOMES:
    if f"{outcome}_w23" in w23_cols:
        outcome_columns.append(f"{outcome}_w23")
    elif f"{outcome}_w2" in combo_dataset.columns:
        outcome_columns.append(f"{outcome}_w2")
    elif f"{outcome}_w3" in combo_dataset.columns:
        outcome_columns.append(f"{outcome}_w3")
    elif outcome in w1_cols:
        outcome_columns.append(outcome)

print(f"Analysis variables selected:")
print(f"  Predictor columns: {len(predictor_columns)}")
print(f"    {predictor_columns}")
print(f"  Outcome columns: {len(outcome_columns)}")
print(f"    {outcome_columns}")

# Create analysis dataset
analysis_columns = ['StFCID'] + predictor_columns + outcome_columns
analysis_dataset = combo_dataset[analysis_columns].copy()

print(f"\nAnalysis dataset shape: {analysis_dataset.shape}")


Creating analysis-ready dataset...
Analysis variables selected:
  Predictor columns: 13
    ['CurrFTE', 'CurrPTE', 'EmplySklls', 'SocSecrty', 'EducAid', 'PubFinAs', 'PubFoodAs', 'PubHousAs', 'OthrFinAs', 'Medicaid', 'OthrHlthIn', 'MedicalIn', 'MentlHlthIn']
  Outcome columns: 6
    ['Homeless_w23', 'SubAbuse_w23', 'Incarc_w23', 'Children_w23', 'Marriage_w23', 'CnctAdult_w23']

Analysis dataset shape: (720, 20)


In [22]:
analysis_dataset.head()

,StFCID,CurrFTE,CurrPTE,EmplySklls,SocSecrty,EducAid,PubFinAs,PubFoodAs,PubHousAs,OthrFinAs,Medicaid,OthrHlthIn,MedicalIn,MentlHlthIn,Homeless_w23,SubAbuse_w23,Incarc_w23,Children_w23,Marriage_w23,CnctAdult_w23
0,TNµ®œ¿ˆ©¬«÷½û¥,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,1,1,1
1,TNµ®œ¿ˆ®¦¥þ¸ý¤,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,1,1,1
2,TNµ®œ¿ˆ®©¯ô¸ù¤,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,0,1,1,1,1,1
3,TNµ®œ¿ˆ®«¨ôºü,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,1,0,0,NaN,1
4,TNµ®œ¿ˆ®«®ð¸ø¬,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,1,0,0,1,1,1


In [23]:
# Cell 6: Check data quality and missing values - preserve NaN as blank cells
print("\nChecking data quality...")
print("="*50)

# First, ensure any 0s that should be NaN are converted back to NaN
print("Step 1: Converting any inappropriate 0s back to NaN...")
for col in predictor_columns + outcome_columns:
    if col in analysis_dataset.columns:
        # Check if column has suspicious patterns (many 0s that might be missing data)
        zero_count = (analysis_dataset[col] == 0).sum()
        total_count = len(analysis_dataset)
        if zero_count > 0:
            print(f"  {col}: {zero_count} zeros found (keeping as-is for now)")

# Missing data analysis - count actual NaN values
print("\nStep 2: Missing data summary (NaN values only):")
missing_data = analysis_dataset.isnull().sum()
missing_pct = (missing_data / len(analysis_dataset) * 100).round(1)

total_missing = 0
for col in predictor_columns + outcome_columns:
    if missing_data[col] > 0:
        total_missing += missing_data[col]
        print(f"  {col}: {missing_data[col]:,} missing ({missing_pct[col]}%) - preserved as NaN")

if total_missing == 0:
    print("  ✅ No missing values found - all data complete")
else:
    print(f"  Total NaN values: {total_missing:,} (preserved as blank cells)")

# Complete cases analysis - only count rows with ALL data present
modeling_columns = predictor_columns + outcome_columns
complete_cases = analysis_dataset.dropna(subset=modeling_columns)

print(f"\nStep 3: Complete cases analysis:")
print(f"  Total records: {len(analysis_dataset):,}")
print(f"  Complete cases (no NaN): {len(complete_cases):,}")
print(f"  Records with missing data: {len(analysis_dataset) - len(complete_cases):,}")
print(f"  Retention rate: {len(complete_cases)/len(analysis_dataset)*100:.1f}%")

# Show sample data with NaN preservation
print(f"\nStep 4: Sample of analysis dataset (NaN preserved):")
sample_data = analysis_dataset.head()
print(sample_data)

# Verify data types - ensure nothing was inappropriately converted
print(f"\nStep 5: Data type verification:")
for col in (predictor_columns + outcome_columns)[:5]:  # Show first 5 columns
    if col in analysis_dataset.columns:
        dtype = analysis_dataset[col].dtype
        nan_count = analysis_dataset[col].isnull().sum()
        print(f"  {col}: {dtype}, {nan_count} NaN values")

print(f"\n✅ Data quality check complete!")
print(f"✅ All missing values preserved as NaN (blank cells)")
print(f"✅ No inappropriate conversion to 0")


Checking data quality...
Step 1: Converting any inappropriate 0s back to NaN...

Step 2: Missing data summary (NaN values only):
  CurrFTE: 10 missing (1.4%) - preserved as NaN
  CurrPTE: 9 missing (1.2%) - preserved as NaN
  EmplySklls: 13 missing (1.8%) - preserved as NaN
  SocSecrty: 12 missing (1.7%) - preserved as NaN
  EducAid: 71 missing (9.9%) - preserved as NaN
  PubFinAs: 598 missing (83.1%) - preserved as NaN
  PubFoodAs: 599 missing (83.2%) - preserved as NaN
  PubHousAs: 599 missing (83.2%) - preserved as NaN
  OthrFinAs: 6 missing (0.8%) - preserved as NaN
  Medicaid: 28 missing (3.9%) - preserved as NaN
  OthrHlthIn: 89 missing (12.4%) - preserved as NaN
  MedicalIn: 678 missing (94.2%) - preserved as NaN
  MentlHlthIn: 680 missing (94.4%) - preserved as NaN
  Homeless_w23: 1 missing (0.1%) - preserved as NaN
  SubAbuse_w23: 3 missing (0.4%) - preserved as NaN
  Incarc_w23: 2 missing (0.3%) - preserved as NaN
  Children_w23: 8 missing (1.1%) - preserved as NaN
  Marriag

In [24]:
# write the first 5 rows (head) to CSV, without the DataFrame index
analysis_dataset.to_csv('analysis_dataset_head.csv')

# Cleaning & Grouping for Stats

##### Preprocess function

In [25]:
# Cell 1: Analyze combo_dataset structure and keep both wave columns
print("Processing combo_dataset - keeping both _w1 and _w23 columns...")
print("="*60)

# Check the combo_dataset structure
print(f"Combo dataset info:")
print(f"  Shape: {combo_dataset.shape}")
print(f"  Total columns: {len(combo_dataset.columns)}")
print(f"  Records: {len(combo_dataset):,}")

# Identify column types by suffix
w1_columns = [col for col in combo_dataset.columns if col.endswith('_w1')]
w23_columns = [col for col in combo_dataset.columns if col.endswith('_w23')]
shared_columns = [col for col in combo_dataset.columns if not col.endswith(('_w1', '_w23'))]

print(f"\nColumn breakdown:")
print(f"  Wave1 columns (_w1): {len(w1_columns)}")
print(f"  Wave23 columns (_w23): {len(w23_columns)}")
print(f"  Shared columns (no suffix): {len(shared_columns)}")

print(f"\nShared columns: {shared_columns}")

# Show sample of paired columns
print(f"\nSample Wave1 columns: {w1_columns[:5]}")
print(f"Sample Wave23 columns: {w23_columns[:5]}")

Processing combo_dataset - keeping both _w1 and _w23 columns...
Combo dataset info:
  Shape: (720, 82)
  Total columns: 82
  Records: 720

Column breakdown:
  Wave1 columns (_w1): 4
  Wave23 columns (_w23): 40
  Shared columns (no suffix): 38

Shared columns: ['StFCID', 'RepDate', 'OutcmRpt', 'OutcmDte', 'OutcmFCS', 'CurrFTE', 'CurrPTE', 'EmplySklls', 'SocSecrty', 'EducAid', 'PubFinAs', 'PubFoodAs', 'PubHousAs', 'OthrFinAs', 'HighEdCert', 'CurrenRoll', 'CnctAdult', 'Homeless', 'SubAbuse', 'Incarc', 'Children', 'Marriage', 'Medicaid', 'OthrHlthIn', 'MedicalIn', 'MentlHlthIn', 'PrescripIn', 'Baseline', 'Elig19', 'Elig21', 'Responded', 'Race', 'HS_or_GED', 'Voc_Certificate', 'Voc_License', 'Assoc_Degree', 'Bach_Degree', 'Higher_Degree']

Sample Wave1 columns: ['StFIPS_w1', 'St_w1', 'Sex_w1', 'Cohort_w1']
Sample Wave23 columns: ['StFIPS_w23', 'St_w23', 'Sex_w23', 'Cohort_w23', 'Elig19_w23']


In [26]:
# Cell 2: Find matching column pairs between waves - FIXED for consistent _w1 naming
print("\nFinding matching column pairs between waves...")
print("="*50)

# Get base names for each wave (remove suffixes)
w1_base_names = [col.replace('_w1', '') for col in w1_columns]
w23_base_names = [col.replace('_w23', '') for col in w23_columns]

# Find which variables exist in both waves
paired_variables = []
w1_only_variables = []
w23_only_variables = []

for base_name in set(w1_base_names + w23_base_names):
    w1_col = f"{base_name}_w1"  # Now ALL Wave1 columns have _w1
    w23_col = f"{base_name}_w23"
    
    has_w1 = w1_col in combo_dataset.columns
    has_w23 = w23_col in combo_dataset.columns
    
    if has_w1 and has_w23:
        paired_variables.append(base_name)
    elif has_w1:
        w1_only_variables.append(base_name)
    elif has_w23:
        w23_only_variables.append(base_name)

print(f"Variable analysis:")
print(f"  Paired variables (both waves): {len(paired_variables)}")
print(f"  Wave1 only variables: {len(w1_only_variables)}")
print(f"  Wave23 only variables: {len(w23_only_variables)}")

print(f"\nPaired variables (first 10): {paired_variables[:10]}")
if w1_only_variables:
    print(f"Wave1 only (first 5): {w1_only_variables[:5]}")
if w23_only_variables:
    print(f"Wave23 only (first 5): {w23_only_variables[:5]}")

# Show example of paired columns
print(f"\nExample paired columns:")
for base_name in paired_variables[:5]:
    w1_name = f"{base_name}_w1"
    w23_name = f"{base_name}_w23"
    print(f"  {base_name}: {w1_name} & {w23_name}")


Finding matching column pairs between waves...
Variable analysis:
  Paired variables (both waves): 4
  Wave1 only variables: 0
  Wave23 only variables: 36

Paired variables (first 10): ['StFIPS', 'St', 'Sex', 'Cohort']
Wave23 only (first 5): ['EmplySklls', 'Elig19', 'Assoc_Degree', 'OutcmRpt', 'Elig21']

Example paired columns:
  StFIPS: StFIPS_w1 & StFIPS_w23
  St: St_w1 & St_w23
  Sex: Sex_w1 & Sex_w23
  Cohort: Cohort_w1 & Cohort_w23


In [27]:
# Cell 3: Create comprehensive clean dataset preserving both waves
print("\nCreating comprehensive clean dataset...")
print("="*50)

# Start with the complete combo_dataset
comprehensive_clean = combo_dataset.copy()

print(f"Starting comprehensive cleaning:")
print(f"  Original shape: {comprehensive_clean.shape}")

# Get all analysis columns (both waves)
all_analysis_columns = w1_columns + w23_columns

# Clean text data but preserve NaN
print(f"\nStep 1: Text cleaning (preserving NaN)...")
for col in all_analysis_columns:
    if col in comprehensive_clean.columns:
        # Only clean non-null values
        mask = comprehensive_clean[col].notna()
        if mask.any():
            comprehensive_clean.loc[mask, col] = comprehensive_clean.loc[mask, col].astype(str).str.strip().str.upper()

# Handle missing indicators
print(f"Step 2: Converting missing indicators to NaN...")
missing_indicators = ['NAN', 'NONE', 'NULL', '', 'MISSING', 'UNKNOWN', 'N/A', 'NA', 'nan']

missing_converted = 0
for col in all_analysis_columns:
    if col in comprehensive_clean.columns:
        before_count = comprehensive_clean[col].isin(missing_indicators).sum()
        comprehensive_clean[col] = comprehensive_clean[col].replace(missing_indicators, np.nan)
        missing_converted += before_count

print(f"  Converted {missing_converted:,} missing indicators to NaN")

# Binary encoding
print(f"Step 3: Binary encoding (preserving NaN)...")
binary_mapping = {
    'YES': 1, 'Y': 1, '1': 1, 'TRUE': 1, 'T': 1,
    'NO': 0, 'N': 0, '0': 0, 'FALSE': 0, 'F': 0
}

encoded_columns = 0
for col in all_analysis_columns:
    if col in comprehensive_clean.columns:
        unique_vals = comprehensive_clean[col].dropna().unique()
        if len(unique_vals) <= 5:  # Likely binary
            before_encoding = comprehensive_clean[col].notna().sum()
            comprehensive_clean[col] = comprehensive_clean[col].map(binary_mapping)
            after_encoding = comprehensive_clean[col].notna().sum()
            
            if after_encoding > 0:
                encoded_columns += 1

print(f"  Applied binary encoding to {encoded_columns} columns")

# Convert to numeric
print(f"Step 4: Converting to numeric (preserving NaN)...")
conversion_nans = 0
for col in all_analysis_columns:
    if col in comprehensive_clean.columns:
        original_nulls = comprehensive_clean[col].isnull().sum()
        comprehensive_clean[col] = pd.to_numeric(comprehensive_clean[col], errors='coerce')
        new_nulls = comprehensive_clean[col].isnull().sum()
        conversion_nans += (new_nulls - original_nulls)

if conversion_nans > 0:
    print(f"  {conversion_nans:,} values became NaN during numeric conversion")

print(f"\nComprehensive cleaning complete!")
print(f"  Final shape: {comprehensive_clean.shape}")


Creating comprehensive clean dataset...
Starting comprehensive cleaning:
  Original shape: (720, 82)

Step 1: Text cleaning (preserving NaN)...
Step 2: Converting missing indicators to NaN...
  Converted 0 missing indicators to NaN
Step 3: Binary encoding (preserving NaN)...
  Applied binary encoding to 34 columns
Step 4: Converting to numeric (preserving NaN)...

Comprehensive cleaning complete!
  Final shape: (720, 82)


In [75]:
# Save to CSV file
comprehensive_clean.to_csv('comprehensive_clean_dataset.csv')

In [28]:
# Cell 4: Create specific analysis datasets
print("\nCreating specific analysis datasets...")
print("="*50)

# Dataset 1: Longitudinal analysis (Wave1 predictors -> Wave23 outcomes)
longitudinal_predictors = [f"{pred}_w1" for pred in PREDICTORS if f"{pred}_w1" in comprehensive_clean.columns]
longitudinal_outcomes = []

for outcome in OUTCOMES:
    if f"{outcome}_w23" in comprehensive_clean.columns:
        longitudinal_outcomes.append(f"{outcome}_w23")

longitudinal_analysis = comprehensive_clean[['StFCID'] + longitudinal_predictors + longitudinal_outcomes].copy()

print(f"Longitudinal analysis dataset:")
print(f"  Shape: {longitudinal_analysis.shape}")
print(f"  Wave1 predictors: {len(longitudinal_predictors)}")
print(f"  Wave23 outcomes: {len(longitudinal_outcomes)}")
print(f"  Predictors: {longitudinal_predictors}")
print(f"  Outcomes: {longitudinal_outcomes}")

# Dataset 2: Complete cases for modeling
complete_cases_longitudinal = longitudinal_analysis.dropna(subset=longitudinal_predictors + longitudinal_outcomes)

print(f"\nComplete cases longitudinal:")
print(f"  Shape: {complete_cases_longitudinal.shape}")
print(f"  Retention rate: {len(complete_cases_longitudinal)/len(comprehensive_clean)*100:.1f}%")

# Dataset 3: All paired variables for comparison analysis
paired_analysis_columns = ['StFCID']
for var in paired_variables:
    w1_col = f"{var}_w1"
    w23_col = f"{var}_w23"
    if w1_col in comprehensive_clean.columns and w23_col in comprehensive_clean.columns:
        paired_analysis_columns.extend([w1_col, w23_col])

paired_analysis = comprehensive_clean[paired_analysis_columns].copy()

print(f"\nPaired variables analysis dataset:")
print(f"  Shape: {paired_analysis.shape}")
print(f"  Paired variables: {len(paired_variables)}")
print(f"  Total paired columns: {len(paired_analysis_columns)-1}")  # Subtract StFCID


Creating specific analysis datasets...
Longitudinal analysis dataset:
  Shape: (720, 7)
  Wave1 predictors: 0
  Wave23 outcomes: 6
  Predictors: []
  Outcomes: ['Homeless_w23', 'SubAbuse_w23', 'Incarc_w23', 'Children_w23', 'Marriage_w23', 'CnctAdult_w23']

Complete cases longitudinal:
  Shape: (188, 7)
  Retention rate: 26.1%

Paired variables analysis dataset:
  Shape: (720, 9)
  Paired variables: 4
  Total paired columns: 8


In [102]:
# Cell 4: Create specific analysis datasets with enhanced quality checks
print("\nCreating specific analysis datasets...")
print("="*50)

# Dataset 1: Longitudinal analysis (Wave1 predictors -> Wave23 outcomes)
longitudinal_predictors = [f"{pred}_w1" for pred in PREDICTORS if f"{pred}_w1" in comprehensive_clean.columns]
longitudinal_outcomes = []
for outcome in OUTCOMES:
    if f"{outcome}_w23" in comprehensive_clean.columns:
        longitudinal_outcomes.append(f"{outcome}_w23")

longitudinal_analysis = comprehensive_clean[['StFCID'] + longitudinal_predictors + longitudinal_outcomes].copy()

print(f"Longitudinal analysis dataset:")
print(f"  Shape: {longitudinal_analysis.shape}")
print(f"  Wave1 predictors: {len(longitudinal_predictors)}")
print(f"  Wave23 outcomes: {len(longitudinal_outcomes)}")
print(f"  Predictors: {longitudinal_predictors}")
print(f"  Outcomes: {longitudinal_outcomes}")

# Enhanced missing data analysis
print(f"\nMissing data patterns in longitudinal dataset:")
predictor_missing = longitudinal_analysis[longitudinal_predictors].isnull().sum()
outcome_missing = longitudinal_analysis[longitudinal_outcomes].isnull().sum()

print(f"Predictors with missing data:")
for pred, missing in predictor_missing[predictor_missing > 0].items():
    print(f"  {pred}: {missing} ({missing/len(longitudinal_analysis)*100:.1f}%)")

print(f"Outcomes with missing data:")
for outcome, missing in outcome_missing[outcome_missing > 0].items():
    print(f"  {outcome}: {missing} ({missing/len(longitudinal_analysis)*100:.1f}%)")

# Dataset 2: Complete cases for modeling
complete_cases_longitudinal = longitudinal_analysis.dropna(subset=longitudinal_predictors + longitudinal_outcomes)
print(f"\nComplete cases longitudinal:")
print(f"  Shape: {complete_cases_longitudinal.shape}")
print(f"  Retention rate: {len(complete_cases_longitudinal)/len(comprehensive_clean)*100:.1f}%")

# Check if retention rate is concerning
retention_rate = len(complete_cases_longitudinal)/len(comprehensive_clean)*100
if retention_rate < 50:
    print(f"  ⚠️  WARNING: Low retention rate ({retention_rate:.1f}%) - consider multiple imputation")
elif retention_rate < 70:
    print(f"  ⚠️  CAUTION: Moderate retention rate ({retention_rate:.1f}%) - check for systematic missingness")
else:
    print(f"  ✓ Good retention rate ({retention_rate:.1f}%)")

# Dataset 3: All paired variables for comparison analysis
paired_analysis_columns = ['StFCID']
valid_paired_vars = []

for var in paired_variables:
    w1_col = f"{var}_w1"
    w23_col = f"{var}_w23"
    if w1_col in comprehensive_clean.columns and w23_col in comprehensive_clean.columns:
        paired_analysis_columns.extend([w1_col, w23_col])
        valid_paired_vars.append(var)

paired_analysis = comprehensive_clean[paired_analysis_columns].copy()

print(f"\nPaired variables analysis dataset:")
print(f"  Shape: {paired_analysis.shape}")
print(f"  Valid paired variables: {len(valid_paired_vars)}")
print(f"  Total paired columns: {len(paired_analysis_columns)-1}")  # Subtract StFCID

# Calculate completeness for each paired variable
print(f"\nPaired variable completeness:")
for var in valid_paired_vars:
    w1_col = f"{var}_w1"
    w23_col = f"{var}_w23"
    both_complete = paired_analysis[[w1_col, w23_col]].dropna().shape[0]
    completeness = both_complete / len(paired_analysis) * 100
    print(f"  {var}: {both_complete}/{len(paired_analysis)} ({completeness:.1f}%)")

# Dataset 4: Additional - Cross-sectional Wave 23 for current status analysis
wave23_columns = ['StFCID'] + [col for col in comprehensive_clean.columns if col.endswith('_w23')]
wave23_analysis = comprehensive_clean[wave23_columns].copy()
print(f"\nWave 23 cross-sectional dataset:")
print(f"  Shape: {wave23_analysis.shape}")
print(f"  Wave 23 variables: {len(wave23_columns)-1}")

# Dataset 5: Additional - Change score dataset
print(f"\nCreating change score dataset...")
change_analysis = comprehensive_clean[['StFCID']].copy()
change_vars_created = 0

for var in valid_paired_vars:
    w1_col = f"{var}_w1"
    w23_col = f"{var}_w23"
    change_col = f"{var}_change"
    
    # Create change score (Wave23 - Wave1)
    change_analysis[change_col] = comprehensive_clean[w23_col] - comprehensive_clean[w1_col]
    change_vars_created += 1

print(f"Change score dataset:")
print(f"  Shape: {change_analysis.shape}")
print(f"  Change variables created: {change_vars_created}")

# Summary statistics
print(f"\n" + "="*50)
print(f"DATASET SUMMARY:")
print(f"="*50)
print(f"1. Longitudinal prediction: {longitudinal_analysis.shape[0]} cases, {len(longitudinal_predictors)} predictors → {len(longitudinal_outcomes)} outcomes")
print(f"2. Complete cases: {complete_cases_longitudinal.shape[0]} cases ({retention_rate:.1f}% retention)")
print(f"3. Paired variables: {paired_analysis.shape[0]} cases, {len(valid_paired_vars)} variable pairs")
print(f"4. Wave 23 cross-sectional: {wave23_analysis.shape[0]} cases, {len(wave23_columns)-1} variables")
print(f"5. Change scores: {change_analysis.shape[0]} cases, {change_vars_created} change variables")

# Store datasets in a dictionary for easy access
datasets = {
    'longitudinal': longitudinal_analysis,
    'complete_longitudinal': complete_cases_longitudinal,
    'paired': paired_analysis,
    'wave23': wave23_analysis,
    'change': change_analysis
}

print(f"\nAll datasets stored in 'datasets' dictionary")
print(f"Access with: datasets['longitudinal'], datasets['complete_longitudinal'], etc.")


Creating specific analysis datasets...
Longitudinal analysis dataset:
  Shape: (720, 20)
  Wave1 predictors: 13
  Wave23 outcomes: 6
  Predictors: ['CurrFTE_w1', 'CurrPTE_w1', 'EmplySklls_w1', 'SocSecrty_w1', 'EducAid_w1', 'PubFinAs_w1', 'PubFoodAs_w1', 'PubHousAs_w1', 'OthrFinAs_w1', 'Medicaid_w1', 'OthrHlthIn_w1', 'MedicalIn_w1', 'MentlHlthIn_w1']
  Outcomes: ['Homeless_w23', 'SubAbuse_w23', 'Incarc_w23', 'Children_w23', 'Marriage_w23', 'CnctAdult_w23']

Missing data patterns in longitudinal dataset:
Predictors with missing data:
  CurrFTE_w1: 720 (100.0%)
  CurrPTE_w1: 720 (100.0%)
  EmplySklls_w1: 720 (100.0%)
  SocSecrty_w1: 720 (100.0%)
  EducAid_w1: 720 (100.0%)
  PubFinAs_w1: 720 (100.0%)
  PubFoodAs_w1: 720 (100.0%)
  PubHousAs_w1: 720 (100.0%)
  OthrFinAs_w1: 720 (100.0%)
  Medicaid_w1: 720 (100.0%)
  OthrHlthIn_w1: 720 (100.0%)
  MedicalIn_w1: 720 (100.0%)
  MentlHlthIn_w1: 720 (100.0%)
Outcomes with missing data:
  Homeless_w23: 1 (0.1%)
  SubAbuse_w23: 3 (0.4%)
  Incarc_w

# Relationships

In [105]:
# Cell 5: Find significant predictor → outcome relationships
print("\nAnalyzing predictor → outcome relationships...")
print("="*60)

import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')

# Check complete cases first, but use pairwise deletion if needed
complete_cases_data = datasets['complete_longitudinal'].copy()
print(f"Complete cases dataset has {complete_cases_data.shape[0]} cases")

if complete_cases_data.shape[0] < 50:  # Not enough complete cases
    print("⚠️  Using pairwise deletion instead of complete cases due to extensive missing data")
    analysis_data = datasets['longitudinal'].copy()
    use_pairwise = True
else:
    analysis_data = complete_cases_data
    use_pairwise = False

print(f"Analyzing {analysis_data.shape[0]} cases (pairwise deletion: {use_pairwise})")

# Separate predictors and outcomes
predictor_cols = [col for col in analysis_data.columns if col.endswith('_w1') and col != 'StFCID']
outcome_cols = [col for col in analysis_data.columns if col.endswith('_w23')]

print(f"Testing {len(predictor_cols)} predictors against {len(outcome_cols)} outcomes")
print(f"Total tests: {len(predictor_cols) * len(outcome_cols)}")

# Diagnostic: Check missing data patterns
print(f"\nMISSING DATA DIAGNOSTIC:")
print(f"="*40)
print(f"Wave 1 predictors missing data:")
for col in predictor_cols:
    missing = analysis_data[col].isnull().sum()
    total = len(analysis_data)
    print(f"  {col.replace('_w1', '')}: {missing}/{total} ({missing/total*100:.1f}%)")

print(f"\nWave 23 outcomes missing data:")
for col in outcome_cols:
    missing = analysis_data[col].isnull().sum()
    total = len(analysis_data)
    print(f"  {col.replace('_w23', '')}: {missing}/{total} ({missing/total*100:.1f}%)")

# Check pairwise availability
print(f"\nPAIRWISE DATA AVAILABILITY (sample sizes for each test):")
print(f"="*60)
pairwise_counts = []
for predictor in predictor_cols:
    for outcome in outcome_cols:
        valid_pairs = (~analysis_data[predictor].isnull() & ~analysis_data[outcome].isnull()).sum()
        pairwise_counts.append(valid_pairs)
        if valid_pairs >= 20:  # Only show pairs with sufficient data
            pred_short = predictor.replace('_w1', '')
            outcome_short = outcome.replace('_w23', '')
            print(f"  {pred_short} → {outcome_short}: N = {valid_pairs}")

if pairwise_counts:
    print(f"\nPairwise sample size summary:")
    print(f"  Range: {min(pairwise_counts)} to {max(pairwise_counts)}")
    print(f"  Mean: {np.mean(pairwise_counts):.1f}")
    print(f"  Pairs with N ≥ 20: {sum(1 for n in pairwise_counts if n >= 20)}")
    print(f"  Pairs with N ≥ 50: {sum(1 for n in pairwise_counts if n >= 50)}")
else:
    print("  No valid pairs found!")

# Storage for results
significant_relationships = []
all_relationships = []

# Function to determine variable type and choose appropriate test
def analyze_relationship(predictor_data, outcome_data, pred_name, outcome_name):
    """Analyze relationship between predictor and outcome variables"""
    
    # Remove any remaining NaN values (pairwise deletion)
    valid_indices = ~(pd.isna(predictor_data) | pd.isna(outcome_data))
    pred_clean = predictor_data[valid_indices]
    outcome_clean = outcome_data[valid_indices]
    
    if len(pred_clean) < 20:  # Need minimum sample size for meaningful analysis
        return {
            'predictor': pred_name,
            'outcome': outcome_name,
            'test': 'Insufficient data',
            'statistic': np.nan,
            'p_value': 1.0,
            'effect_size': 0,
            'n_cases': len(pred_clean),
            'pred_unique_values': 0,
            'outcome_unique_values': 0,
            'insufficient_n': True
        }
    
    # Determine if variables are continuous or categorical
    pred_unique = len(pred_clean.unique())
    outcome_unique = len(outcome_clean.unique())
    
    # Choose appropriate statistical test
    if pred_unique <= 5 and outcome_unique <= 5:
        # Both categorical - Chi-square test
        try:
            contingency_table = pd.crosstab(pred_clean, outcome_clean)
            if contingency_table.size > 1:
                chi2, p_value = stats.chi2_contingency(contingency_table)[:2]
                test_used = "Chi-square"
                statistic = chi2
                effect_size = np.sqrt(chi2 / (len(pred_clean) * (min(contingency_table.shape) - 1)))  # Cramér's V
            else:
                return None
        except:
            return None
            
    elif pred_unique <= 5 and outcome_unique > 5:
        # Predictor categorical, outcome continuous - ANOVA/t-test
        try:
            groups = [outcome_clean[pred_clean == group] for group in pred_clean.unique()]
            groups = [g for g in groups if len(g) > 0]  # Remove empty groups
            
            if len(groups) == 2:
                # t-test
                statistic, p_value = stats.ttest_ind(groups[0], groups[1])
                test_used = "t-test"
                # Cohen's d effect size
                pooled_std = np.sqrt((groups[0].var() + groups[1].var()) / 2)
                effect_size = abs((groups[0].mean() - groups[1].mean()) / pooled_std) if pooled_std > 0 else 0
            elif len(groups) > 2:
                # One-way ANOVA
                statistic, p_value = stats.f_oneway(*groups)
                test_used = "ANOVA"
                # Eta-squared effect size
                overall_mean = outcome_clean.mean()
                ss_between = sum(len(g) * (g.mean() - overall_mean)**2 for g in groups)
                ss_total = sum((outcome_clean - overall_mean)**2)
                effect_size = ss_between / ss_total if ss_total > 0 else 0
            else:
                return None
        except:
            return None
            
    elif pred_unique > 5 and outcome_unique <= 5:
        # Predictor continuous, outcome categorical - reverse ANOVA
        try:
            groups = [pred_clean[outcome_clean == group] for group in outcome_clean.unique()]
            groups = [g for g in groups if len(g) > 0]
            
            if len(groups) == 2:
                statistic, p_value = stats.ttest_ind(groups[0], groups[1])
                test_used = "t-test (reversed)"
                pooled_std = np.sqrt((groups[0].var() + groups[1].var()) / 2)
                effect_size = abs((groups[0].mean() - groups[1].mean()) / pooled_std) if pooled_std > 0 else 0
            elif len(groups) > 2:
                statistic, p_value = stats.f_oneway(*groups)
                test_used = "ANOVA (reversed)"
                overall_mean = pred_clean.mean()
                ss_between = sum(len(g) * (g.mean() - overall_mean)**2 for g in groups)
                ss_total = sum((pred_clean - overall_mean)**2)
                effect_size = ss_between / ss_total if ss_total > 0 else 0
            else:
                return None
        except:
            return None
            
    else:
        # Both continuous - Pearson correlation (with Spearman as backup)
        try:
            # Try Pearson first
            r_pearson, p_pearson = pearsonr(pred_clean, outcome_clean)
            
            # Also calculate Spearman for robustness
            r_spearman, p_spearman = spearmanr(pred_clean, outcome_clean)
            
            # Use Pearson if assumptions are reasonable, otherwise Spearman
            if abs(r_pearson) > abs(r_spearman):
                statistic, p_value = r_pearson, p_pearson
                test_used = "Pearson correlation"
            else:
                statistic, p_value = r_spearman, p_spearman
                test_used = "Spearman correlation"
                
            effect_size = abs(statistic)  # Correlation coefficient is the effect size
            
        except:
            return None
    
    return {
        'predictor': pred_name,
        'outcome': outcome_name,
        'test': test_used,
        'statistic': statistic,
        'p_value': p_value,
        'effect_size': effect_size,
        'n_cases': len(pred_clean),
        'pred_unique_values': pred_unique,
        'outcome_unique_values': outcome_unique
    }

# Run analysis for all predictor-outcome pairs
print("\nRunning statistical tests...")
print("Progress: ", end="", flush=True)

total_tests = len(predictor_cols) * len(outcome_cols)
completed_tests = 0

for i, predictor in enumerate(predictor_cols):
    for j, outcome in enumerate(outcome_cols):
        
        # Progress indicator
        if completed_tests % max(1, total_tests // 20) == 0:
            print("█", end="", flush=True)
        
        result = analyze_relationship(
            analysis_data[predictor], 
            analysis_data[outcome], 
            predictor, 
            outcome
        )
        
        if result is not None:
            all_relationships.append(result)
            
            # Store significant relationships (p < 0.05) with sufficient data
            if result['p_value'] < 0.05 and not result.get('insufficient_n', False):
                significant_relationships.append(result)
        
        completed_tests += 1

print(f" Complete!")

# Convert to DataFrames for easier analysis
all_results_df = pd.DataFrame(all_relationships)
significant_results_df = pd.DataFrame(significant_relationships)

# Check data availability
if len(all_relationships) > 0:
    insufficient_data_count = sum(1 for r in all_relationships if r.get('insufficient_n', False))
    valid_tests = len(all_relationships) - insufficient_data_count
    
    print(f"\n" + "="*60)
    print(f"DATA AVAILABILITY:")
    print(f"="*60)
    print(f"Total predictor-outcome pairs: {len(predictor_cols) * len(outcome_cols)}")
    print(f"Pairs with insufficient data (N < 20): {insufficient_data_count}")
    print(f"Pairs with sufficient data: {valid_tests}")
    
    if valid_tests > 0:
        # Show sample sizes for valid tests
        valid_relationships = [r for r in all_relationships if not r.get('insufficient_n', False)]
        sample_sizes = [r['n_cases'] for r in valid_relationships]
        print(f"Sample size range: {min(sample_sizes)} to {max(sample_sizes)} cases")
        print(f"Mean sample size: {np.mean(sample_sizes):.1f}")
    
print(f"\n" + "="*60)
print(f"RESULTS SUMMARY:")
print(f"="*60)
print(f"Total valid tests conducted: {len(all_relationships)}")
print(f"Significant relationships (p < 0.05): {len(significant_relationships)}")
print(f"Significance rate: {len(significant_relationships)/len(all_relationships)*100:.1f}%")

# Apply multiple comparison correction (Bonferroni)
if len(all_relationships) > 0:
    alpha_bonferroni = 0.05 / len(all_relationships)
    bonferroni_significant = [r for r in significant_relationships if r['p_value'] < alpha_bonferroni]
    print(f"Bonferroni corrected significant (α = {alpha_bonferroni:.6f}): {len(bonferroni_significant)}")
    
    # Also try False Discovery Rate (FDR) correction - less conservative
    from statsmodels.stats.multitest import multipletests
    if len(all_relationships) > 0:
        p_values = [r['p_value'] for r in all_relationships]
        fdr_rejected, fdr_p_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')
        fdr_significant_count = sum(fdr_rejected)
        print(f"FDR corrected significant (α = 0.05): {fdr_significant_count}")

# Display top significant relationships
if len(significant_relationships) > 0:
    print(f"\n" + "="*60)
    print(f"TOP SIGNIFICANT RELATIONSHIPS:")
    print(f"="*60)
    
    # Sort by p-value
    sorted_significant = sorted(significant_relationships, key=lambda x: x['p_value'])
    
    for i, result in enumerate(sorted_significant[:15]):  # Show top 15
        pred_clean = result['predictor'].replace('_w1', '')
        outcome_clean = result['outcome'].replace('_w23', '')
        
        print(f"{i+1:2d}. {pred_clean} → {outcome_clean}")
        print(f"    {result['test']}: {result['statistic']:.4f}, p = {result['p_value']:.6f}")
        print(f"    Effect size: {result['effect_size']:.4f}, N = {result['n_cases']}")
        
        # Add interpretation for effect size
        if 'correlation' in result['test'].lower():
            if result['effect_size'] >= 0.5:
                size_interpretation = "(large effect)"
            elif result['effect_size'] >= 0.3:
                size_interpretation = "(medium effect)"
            elif result['effect_size'] >= 0.1:
                size_interpretation = "(small effect)"
            else:
                size_interpretation = "(very small effect)"
        else:
            if result['effect_size'] >= 0.8:
                size_interpretation = "(large effect)"
            elif result['effect_size'] >= 0.5:
                size_interpretation = "(medium effect)"
            elif result['effect_size'] >= 0.2:
                size_interpretation = "(small effect)"
            else:
                size_interpretation = "(very small effect)"
        
        print(f"    {size_interpretation}")
        print()

# Summary by test type
if len(significant_relationships) > 0:
    print(f"BREAKDOWN BY TEST TYPE:")
    print(f"="*30)
    test_counts = {}
    for result in significant_relationships:
        test_type = result['test']
        test_counts[test_type] = test_counts.get(test_type, 0) + 1
    
    for test_type, count in sorted(test_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{test_type}: {count}")

# Store results for future use
results_summary = {
    'all_results': all_results_df,
    'significant_results': significant_results_df,
    'bonferroni_significant': bonferroni_significant if len(all_relationships) > 0 else [],
    'alpha_bonferroni': alpha_bonferroni if len(all_relationships) > 0 else None,
    'fdr_significant_count': fdr_significant_count if len(all_relationships) > 0 else 0
}

print(f"\nResults stored in 'results_summary' dictionary")
print(f"Access with: results_summary['significant_results'], etc.")

# Save significant results to CSV for external analysis
if len(significant_relationships) > 0:
    significant_results_df.to_csv('significant_predictor_outcome_relationships.csv', index=False)
    print(f"\nSignificant results saved to 'significant_predictor_outcome_relationships.csv'")
else:
    print(f"\nNo significant relationships found to save.")


Analyzing predictor → outcome relationships...
Complete cases dataset has 0 cases
⚠️  Using pairwise deletion instead of complete cases due to extensive missing data
Analyzing 720 cases (pairwise deletion: True)
Testing 13 predictors against 6 outcomes
Total tests: 78

MISSING DATA DIAGNOSTIC:
Wave 1 predictors missing data:
  CurrFTE: 720/720 (100.0%)
  CurrPTE: 720/720 (100.0%)
  EmplySklls: 720/720 (100.0%)
  SocSecrty: 720/720 (100.0%)
  EducAid: 720/720 (100.0%)
  PubFinAs: 720/720 (100.0%)
  PubFoodAs: 720/720 (100.0%)
  PubHousAs: 720/720 (100.0%)
  OthrFinAs: 720/720 (100.0%)
  Medicaid: 720/720 (100.0%)
  OthrHlthIn: 720/720 (100.0%)
  MedicalIn: 720/720 (100.0%)
  MentlHlthIn: 720/720 (100.0%)

Wave 23 outcomes missing data:
  Homeless: 1/720 (0.1%)
  SubAbuse: 3/720 (0.4%)
  Incarc: 2/720 (0.3%)
  Children: 8/720 (1.1%)
  Marriage: 531/720 (73.8%)
  CnctAdult: 1/720 (0.1%)

PAIRWISE DATA AVAILABILITY (sample sizes for each test):

Pairwise sample size summary:
  Range: 0 to

# Visual Models

#### Outcome = 1 for significant

In [106]:
def create_relationship_plots(significant_df, data_df, max_plots=12):
    """Create bar plots showing how Wave 1 predictors affect Wave 2-3 outcome probabilities"""
    
    if len(significant_df) == 0:
        print("No significant relationships to plot")
        return
    
    # Take the most significant relationships
    top_relationships = significant_df.nsmallest(max_plots, 'P_Value')
    
    # Calculate number of rows and columns for subplots
    n_plots = len(top_relationships)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (_, row) in enumerate(top_relationships.iterrows()):
        predictor = row['Predictor']
        outcome = row['Outcome']
        p_value = row['P_Value']
        odds_ratio = row['Odds_Ratio']
        
        # Calculate subplot position
        row_idx = idx // n_cols
        col_idx = idx % n_cols
        ax = axes[row_idx, col_idx]
        
        # Get clean data for this relationship
        plot_data = data_df[[predictor, outcome]].dropna()
        
        if len(plot_data) > 0:
            # Calculate outcome probabilities by predictor value
            prob_table = plot_data.groupby(predictor)[outcome].agg(['mean', 'count']).reset_index()
            prob_table.columns = [predictor, 'outcome_probability', 'sample_size']
            
            # Create labels
            labels = [f"{predictor}=0\n(n={prob_table[prob_table[predictor]==0]['sample_size'].iloc[0] if 0 in prob_table[predictor].values else 0})",
                     f"{predictor}=1\n(n={prob_table[prob_table[predictor]==1]['sample_size'].iloc[0] if 1 in prob_table[predictor].values else 0})"]
            
            # Get probabilities
            prob_0 = prob_table[prob_table[predictor]==0]['outcome_probability'].iloc[0] if 0 in prob_table[predictor].values else 0
            prob_1 = prob_table[prob_table[predictor]==1]['outcome_probability'].iloc[0] if 1 in prob_table[predictor].values else 0
            
            probabilities = [prob_0, prob_1]
            
            # Create bar plot
            bars = ax.bar([0, 1], probabilities, color=['lightblue', 'orange'], alpha=0.7)
            
            # Add percentage labels on bars
            for i, (bar, prob) in enumerate(zip(bars, probabilities)):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{prob:.1%}', ha='center', va='bottom', fontweight='bold')
            
            # Clean outcome name for title
            clean_outcome = outcome.replace('_w23', '')
            ax.set_title(f'{predictor} → {clean_outcome}\nOR={odds_ratio:.2f}, p={p_value:.3f}', fontsize=10)
            ax.set_ylabel(f'Probability of {clean_outcome}=1')
            ax.set_xticks([0, 1])
            ax.set_xticklabels(labels)
            ax.set_ylim(0, max(probabilities) * 1.2)
            
        else:
            ax.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax.transAxes)
    
    # Hide empty subplots
    for idx in range(n_plots, n_rows * n_cols):
        row_idx = idx // n_cols
        col_idx = idx % n_cols
        axes[row_idx, col_idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Create the visualization
create_relationship_plots(longitudinal_sig, longitudinal_clean)

NameError: name 'longitudinal_sig' is not defined

#### Outcome = 0 for significant

In [107]:
def create_relationship_plots(significant_df, data_df, max_plots=12):
    """Create bar plots showing how Wave 1 predictors affect Wave 2-3 outcome probabilities"""
    
    if len(significant_df) == 0:
        print("No significant relationships to plot")
        return
    
    # Take the most significant relationships
    top_relationships = significant_df.nsmallest(max_plots, 'P_Value')
    
    # Calculate number of rows and columns for subplots
    n_plots = len(top_relationships)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (_, row) in enumerate(top_relationships.iterrows()):
        predictor = row['Predictor']
        outcome = row['Outcome']
        p_value = row['P_Value']
        odds_ratio = row['Odds_Ratio']
        
        # Calculate subplot position
        row_idx = idx // n_cols
        col_idx = idx % n_cols
        ax = axes[row_idx, col_idx]
        
        # Get clean data for this relationship
        plot_data = data_df[[predictor, outcome]].dropna()
        
        if len(plot_data) > 0:
            # Calculate outcome probabilities by predictor value
            prob_table = plot_data.groupby(predictor)[outcome].agg(['mean', 'count']).reset_index()
            prob_table.columns = [predictor, 'outcome_probability', 'sample_size']
            
            # Create labels
            labels = [f"{predictor}=0\n(n={prob_table[prob_table[predictor]==0]['sample_size'].iloc[0] if 0 in prob_table[predictor].values else 0})",
                     f"{predictor}=1\n(n={prob_table[prob_table[predictor]==1]['sample_size'].iloc[0] if 1 in prob_table[predictor].values else 0})"]
            
            # Get probabilities for outcome=0 (1 - mean gives probability of 0)
            prob_0 = 1 - prob_table[prob_table[predictor]==0]['outcome_probability'].iloc[0] if 0 in prob_table[predictor].values else 1
            prob_1 = 1 - prob_table[prob_table[predictor]==1]['outcome_probability'].iloc[0] if 1 in prob_table[predictor].values else 1
            
            probabilities = [prob_0, prob_1]
            
            # Create bar plot
            bars = ax.bar([0, 1], probabilities, color=['lightblue', 'orange'], alpha=0.7)
            
            # Add percentage labels on bars
            for i, (bar, prob) in enumerate(zip(bars, probabilities)):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{prob:.1%}', ha='center', va='bottom', fontweight='bold')
            
            # Clean outcome name for title
            clean_outcome = outcome.replace('_w23', '')
            ax.set_title(f'{predictor} → {clean_outcome}\nOR={odds_ratio:.2f}, p={p_value:.3f}', fontsize=10)
            ax.set_ylabel(f'Probability of {clean_outcome}=0')
            ax.set_xticks([0, 1])
            ax.set_xticklabels(labels)
            ax.set_ylim(0, max(probabilities) * 1.2)
            
        else:
            ax.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax.transAxes)
    
    # Hide empty subplots
    for idx in range(n_plots, n_rows * n_cols):
        row_idx = idx // n_cols
        col_idx = idx % n_cols
        axes[row_idx, col_idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Create the visualization
create_relationship_plots(longitudinal_sig, longitudinal_clean)

NameError: name 'longitudinal_sig' is not defined

#### Visual of both

In [108]:
def create_relationship_plots(significant_df, data_df, max_plots=12):
    """Create bar plots showing how Wave 1 predictors affect Wave 2-3 outcome probabilities"""
    
    if len(significant_df) == 0:
        print("No significant relationships to plot")
        return
    
    # Take the most significant relationships
    top_relationships = significant_df.nsmallest(max_plots, 'P_Value')
    
    # Calculate number of rows and columns for subplots
    n_plots = len(top_relationships)
    n_cols = 3
    n_rows = (n_plots + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (_, row) in enumerate(top_relationships.iterrows()):
        predictor = row['Predictor']
        outcome = row['Outcome']
        p_value = row['P_Value']
        odds_ratio = row['Odds_Ratio']
        
        # Calculate subplot position
        row_idx = idx // n_cols
        col_idx = idx % n_cols
        ax = axes[row_idx, col_idx]
        
        # Get clean data for this relationship
        plot_data = data_df[[predictor, outcome]].dropna()
        
        if len(plot_data) > 0:
            # Calculate outcome probabilities by predictor value
            prob_table = plot_data.groupby(predictor)[outcome].agg(['mean', 'count']).reset_index()
            prob_table.columns = [predictor, 'outcome_probability', 'sample_size']
            
            # Get probabilities for outcome=0 and outcome=1
            prob_0_when_pred_0 = prob_table[prob_table[predictor]==0]['outcome_probability'].iloc[0] if 0 in prob_table[predictor].values else 0
            prob_1_when_pred_0 = 1 - prob_0_when_pred_0
            
            prob_0_when_pred_1 = prob_table[prob_table[predictor]==1]['outcome_probability'].iloc[0] if 1 in prob_table[predictor].values else 0
            prob_1_when_pred_1 = 1 - prob_0_when_pred_1
            
            # Sample sizes
            n_pred_0 = prob_table[prob_table[predictor]==0]['sample_size'].iloc[0] if 0 in prob_table[predictor].values else 0
            n_pred_1 = prob_table[prob_table[predictor]==1]['sample_size'].iloc[0] if 1 in prob_table[predictor].values else 0
            
            # Set up grouped bars
            x_positions = [0, 1]  # predictor = 0, predictor = 1
            width = 0.35
            
            # Probabilities for outcome=0 and outcome=1
            prob_outcome_0 = [1 - prob_0_when_pred_0, 1 - prob_0_when_pred_1]  # P(outcome=0)
            prob_outcome_1 = [prob_0_when_pred_0, prob_0_when_pred_1]          # P(outcome=1)
            
            # Create grouped bars
            bars1 = ax.bar([x - width/2 for x in x_positions], prob_outcome_0, width, 
                          label='Outcome=0', color='lightcoral', alpha=0.7)
            bars2 = ax.bar([x + width/2 for x in x_positions], prob_outcome_1, width,
                          label='Outcome=1', color='lightblue', alpha=0.7)
            
            # Add percentage labels on bars
            for bars, probs in [(bars1, prob_outcome_0), (bars2, prob_outcome_1)]:
                for bar, prob in zip(bars, probs):
                    height = bar.get_height()
                    if height > 0.05:  # Only show label if bar is tall enough
                        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                               f'{prob:.1%}', ha='center', va='bottom', fontweight='bold', fontsize=8)
            
            # Clean outcome name for title
            clean_outcome = outcome.replace('_w23', '')
            ax.set_title(f'{predictor} → {clean_outcome}\nOR={odds_ratio:.2f}, p={p_value:.3f}', fontsize=10)
            ax.set_ylabel(f'Probability')
            ax.set_xticks(x_positions)
            ax.set_xticklabels([f'{predictor}=0\n(n={n_pred_0})', f'{predictor}=1\n(n={n_pred_1})'])
            ax.set_ylim(0, 1.1)
            ax.legend(fontsize=8)
            
            # Add grid for easier reading
            ax.grid(True, alpha=0.3, axis='y')
            
        else:
            ax.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax.transAxes)
    
    # Hide empty subplots
    for idx in range(n_plots, n_rows * n_cols):
        row_idx = idx // n_cols
        col_idx = idx % n_cols
        axes[row_idx, col_idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Create the visualization
create_relationship_plots(longitudinal_sig, longitudinal_clean)

NameError: name 'longitudinal_sig' is not defined

#### Re-run stats

In [ ]:
# Cell: Step 1 - Binary Pattern Analysis for 1's and 0's Relationships
def analyze_binary_patterns(data, predictors, outcomes, analysis_name):
    """
    Analyze specific patterns of binary relationships:
    - Predictor=1 & Outcome=1
    - Predictor=1 & Outcome=0  
    - Predictor=0 & Outcome=1
    - Predictor=0 & Outcome=0
    """
    
    pattern_results = []
    
    print(f"\n=== Binary Pattern Analysis - {analysis_name} ===")
    
    for predictor in predictors:
        for outcome in outcomes:
            if predictor not in data.columns or outcome not in data.columns:
                continue
                
            # Create subset with no missing values
            subset = data[[predictor, outcome]].dropna()
            
            if len(subset) < 10:
                continue
                
            # Ensure both variables are binary (0 or 1)
            if set(subset[predictor].unique()) - {0, 1} or set(subset[outcome].unique()) - {0, 1}:
                continue
            
            try:
                # Create crosstab
                crosstab = pd.crosstab(subset[predictor], subset[outcome], margins=True)
                
                if crosstab.shape[0] < 3 or crosstab.shape[1] < 3:  # Need 2x2 + margins
                    continue
                
                # Calculate specific pattern counts and percentages
                total_n = len(subset)
                
                # Pattern counts
                pred0_out0 = len(subset[(subset[predictor] == 0) & (subset[outcome] == 0)])
                pred0_out1 = len(subset[(subset[predictor] == 0) & (subset[outcome] == 1)])
                pred1_out0 = len(subset[(subset[predictor] == 1) & (subset[outcome] == 0)])
                pred1_out1 = len(subset[(subset[predictor] == 1) & (subset[outcome] == 1)])
                
                # Pattern percentages
                pred0_out0_pct = (pred0_out0 / total_n) * 100 if total_n > 0 else 0
                pred0_out1_pct = (pred0_out1 / total_n) * 100 if total_n > 0 else 0
                pred1_out0_pct = (pred1_out0 / total_n) * 100 if total_n > 0 else 0
                pred1_out1_pct = (pred1_out1 / total_n) * 100 if total_n > 0 else 0
                
                # Conditional probabilities
                pred0_total = pred0_out0 + pred0_out1
                pred1_total = pred1_out0 + pred1_out1
                
                prob_out1_given_pred0 = pred0_out1 / pred0_total if pred0_total > 0 else 0
                prob_out1_given_pred1 = pred1_out1 / pred1_total if pred1_total > 0 else 0
                prob_out0_given_pred0 = pred0_out0 / pred0_total if pred0_total > 0 else 0
                prob_out0_given_pred1 = pred1_out0 / pred1_total if pred1_total > 0 else 0
                
                # Chi-square test
                chi2, p_value, dof, expected = chi2_contingency(crosstab.iloc[:-1, :-1])
                
                # Determine dominant pattern
                max_pattern_pct = max(pred0_out0_pct, pred0_out1_pct, pred1_out0_pct, pred1_out1_pct)
                if max_pattern_pct == pred1_out1_pct:
                    dominant_pattern = "Pred=1 & Out=1"
                elif max_pattern_pct == pred1_out0_pct:
                    dominant_pattern = "Pred=1 & Out=0"
                elif max_pattern_pct == pred0_out1_pct:
                    dominant_pattern = "Pred=0 & Out=1"
                else:
                    dominant_pattern = "Pred=0 & Out=0"
                
                pattern_results.append({
                    'Analysis': analysis_name,
                    'Predictor': predictor,
                    'Outcome': outcome,
                    'Total_N': total_n,
                    'Pred0_Out0_Count': pred0_out0,
                    'Pred0_Out0_Pct': round(pred0_out0_pct, 2),
                    'Pred0_Out1_Count': pred0_out1,
                    'Pred0_Out1_Pct': round(pred0_out1_pct, 2),
                    'Pred1_Out0_Count': pred1_out0,
                    'Pred1_Out0_Pct': round(pred1_out0_pct, 2),
                    'Pred1_Out1_Count': pred1_out1,
                    'Pred1_Out1_Pct': round(pred1_out1_pct, 2),
                    'Prob_Out1_Given_Pred0': round(prob_out1_given_pred0, 3),
                    'Prob_Out1_Given_Pred1': round(prob_out1_given_pred1, 3),
                    'Prob_Out0_Given_Pred0': round(prob_out0_given_pred0, 3),
                    'Prob_Out0_Given_Pred1': round(prob_out0_given_pred1, 3),
                    'Dominant_Pattern': dominant_pattern,
                    'Chi2_P_Value': round(p_value, 4),
                    'Significant': p_value < 0.05
                })
                
            except Exception as e:
                print(f"Error in pattern analysis {predictor} -> {outcome}: {str(e)}")
                continue
    
    return pd.DataFrame(pattern_results)


In [ ]:
# Run binary pattern analysis on longitudinal data
longitudinal_patterns = analyze_binary_patterns(longitudinal_clean, long_predictors, long_outcomes, "Longitudinal_Patterns")

# Filter for significant patterns
significant_patterns = longitudinal_patterns[longitudinal_patterns['Significant'] == True].copy()
significant_patterns = significant_patterns.sort_values('Chi2_P_Value')

print(f"\n=== SIGNIFICANT BINARY PATTERNS (p < 0.05) ===")
print(f"Found {len(significant_patterns)} significant binary patterns")

if len(significant_patterns) > 0:
    pattern_display_cols = ['Predictor', 'Outcome', 'Dominant_Pattern', 'Total_N', 
                           'Pred1_Out1_Pct', 'Pred1_Out0_Pct', 'Pred0_Out1_Pct', 'Pred0_Out0_Pct',
                           'Prob_Out1_Given_Pred1', 'Prob_Out1_Given_Pred0', 'Chi2_P_Value']
    print("\nSignificant Binary Patterns:")
    print(significant_patterns[pattern_display_cols])


In [ ]:
# Save results
longitudinal_patterns.to_csv('binary_pattern_analysis.csv', index=False)
significant_patterns.to_csv('significant_binary_patterns.csv', index=False)
print(f"\nPattern analysis saved to CSV files")

#### Higher level 

In [ ]:
# Cell: Step 2 - Wave1 Predictors Causing Higher Levels in Wave23 Outcomes
def analyze_wave1_to_wave23_increases(longitudinal_data, wave1_predictors, wave23_outcomes):
    """
    Analyze how Wave1 predictors lead to higher levels in Wave23 outcomes
    Focus on increases/improvements from Wave1 to Wave23
    """
    
    increase_results = []
    
    print(f"\n=== Wave1 → Wave23 Higher Level Analysis ===")
    
    for predictor in wave1_predictors:
        for outcome in wave23_outcomes:
            if predictor not in longitudinal_data.columns or outcome not in longitudinal_data.columns:
                continue
                
            # Create subset with no missing values
            subset = longitudinal_data[[predictor, outcome]].dropna()
            
            if len(subset) < 10:
                continue
            
            try:
                # Group by predictor and calculate outcome statistics
                grouped_stats = subset.groupby(predictor)[outcome].agg(['count', 'mean', 'std', 'median']).reset_index()
                
                if len(grouped_stats) < 2:
                    continue
                
                # Calculate effect sizes and statistical tests
                pred_0_group = subset[subset[predictor] == 0][outcome]
                pred_1_group = subset[subset[predictor] == 1][outcome]
                
                if len(pred_0_group) < 5 or len(pred_1_group) < 5:
                    continue
                
                # Mean difference (higher is better)
                mean_diff = pred_1_group.mean() - pred_0_group.mean()
                
                # Percentage increase
                pct_increase = ((pred_1_group.mean() - pred_0_group.mean()) / pred_0_group.mean() * 100) if pred_0_group.mean() != 0 else 0
                
                # T-test for mean differences
                from scipy.stats import ttest_ind
                t_stat, t_p_value = ttest_ind(pred_1_group, pred_0_group)
                
                # Mann-Whitney U test (non-parametric)
                from scipy.stats import mannwhitneyu
                try:
                    u_stat, u_p_value = mannwhitneyu(pred_1_group, pred_0_group, alternative='two-sided')
                except:
                    u_p_value = np.nan
                
                # Effect size (Cohen's d)
                pooled_std = np.sqrt(((len(pred_0_group) - 1) * pred_0_group.std()**2 + 
                                    (len(pred_1_group) - 1) * pred_1_group.std()**2) / 
                                   (len(pred_0_group) + len(pred_1_group) - 2))
                cohens_d = mean_diff / pooled_std if pooled_std > 0 else 0
                
                # Determine effect magnitude
                if abs(cohens_d) < 0.2:
                    effect_magnitude = "Small"
                elif abs(cohens_d) < 0.5:
                    effect_magnitude = "Small-Medium"
                elif abs(cohens_d) < 0.8:
                    effect_magnitude = "Medium-Large"
                else:
                    effect_magnitude = "Large"
                
                # Positive effect indicator
                positive_effect = mean_diff > 0
                
                increase_results.append({
                    'Predictor_Wave1': predictor,
                    'Outcome_Wave23': outcome,
                    'Total_N': len(subset),
                    'Pred0_N': len(pred_0_group),
                    'Pred1_N': len(pred_1_group),
                    'Pred0_Mean': round(pred_0_group.mean(), 3),
                    'Pred1_Mean': round(pred_1_group.mean(), 3),
                    'Mean_Difference': round(mean_diff, 3),
                    'Percent_Increase': round(pct_increase, 2),
                    'Cohens_D': round(cohens_d, 3),
                    'Effect_Magnitude': effect_magnitude,
                    'Positive_Effect': positive_effect,
                    'T_Test_P_Value': round(t_p_value, 4),
                    'Mann_Whitney_P_Value': round(u_p_value, 4) if not np.isnan(u_p_value) else np.nan,
                    'T_Test_Significant': t_p_value < 0.05,
                    'MW_Test_Significant': u_p_value < 0.05 if not np.isnan(u_p_value) else False,
                    'Overall_Significant': t_p_value < 0.05 or (u_p_value < 0.05 if not np.isnan(u_p_value) else False)
                })
                
            except Exception as e:
                print(f"Error in increase analysis {predictor} -> {outcome}: {str(e)}")
                continue
    
    return pd.DataFrame(increase_results)

In [ ]:
# Run Wave1 → Wave23 increase analysis
wave1_to_wave23_increases = analyze_wave1_to_wave23_increases(longitudinal_clean, long_predictors, long_outcomes)

# Filter for significant increases
significant_increases = wave1_to_wave23_increases[wave1_to_wave23_increases['Overall_Significant'] == True].copy()

# Sort by effect size (largest positive effects first)
significant_increases = significant_increases.sort_values(['Positive_Effect', 'Cohens_D'], ascending=[False, False])

print(f"\n=== SIGNIFICANT WAVE1 → WAVE23 INCREASES ===")
print(f"Found {len(significant_increases)} significant relationships")

if len(significant_increases) > 0:
    increase_display_cols = ['Predictor_Wave1', 'Outcome_Wave23', 'Mean_Difference', 'Percent_Increase', 
                            'Cohens_D', 'Effect_Magnitude', 'Positive_Effect', 'T_Test_P_Value', 'Total_N']
    print("\nSignificant Wave1 → Wave23 Increases:")
    print(significant_increases[increase_display_cols])

In [ ]:
# Save results
wave1_to_wave23_increases.to_csv('wave1_to_wave23_increases.csv', index=False)
significant_increases.to_csv('significant_wave1_to_wave23_increases.csv', index=False)
print(f"\nWave1 → Wave23 analysis saved to CSV files")

### Multiple way predictor

In [ ]:
# Cell: Step 3 - REWRITTEN Multiple Predictor Analysis
def multiple_predictor_analysis(data, predictors, outcomes, analysis_name, max_predictors=5):
    """
    Rewritten multiple predictor analysis with robust error handling
    """
    warnings.filterwarnings('ignore')
    
    # Initialize results list
    multi_results = []
    
    print(f"\n=== Multiple Predictor Analysis - {analysis_name} ===")
    print(f"Testing {len(predictors)} predictors against {len(outcomes)} outcomes")
    
    # Process each outcome
    for outcome_idx, outcome in enumerate(outcomes):
        if outcome not in data.columns:
            print(f"Outcome {outcome} not found in data columns")
            continue
            
        print(f"\n[{outcome_idx+1}/{len(outcomes)}] Analyzing outcome: {outcome}")
        
        # Get available predictors (excluding the outcome itself)
        available_predictors = [p for p in predictors if p in data.columns and p != outcome]
        
        if len(available_predictors) < 2:
            print(f"  Not enough predictors available: {len(available_predictors)}")
            continue
        
        print(f"  Available predictors: {len(available_predictors)}")
        
        # Create clean dataset
        required_cols = available_predictors + [outcome]
        subset = data[required_cols].dropna()
        
        if len(subset) < 30:
            print(f"  Insufficient data after cleaning: {len(subset)} rows")
            continue
            
        # Check outcome distribution
        outcome_counts = subset[outcome].value_counts()
        if len(outcome_counts) != 2:
            print(f"  Non-binary outcome: {outcome_counts.to_dict()}")
            continue
            
        if min(outcome_counts) < 10:
            print(f"  Insufficient minority class: {outcome_counts.to_dict()}")
            continue
            
        print(f"  Clean dataset: {len(subset)} rows, outcome distribution: {outcome_counts.to_dict()}")
        
        # Prepare features and target
        X = subset[available_predictors]
        y = subset[outcome]
        
        # Standardize features
        try:
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
        except Exception as e:
            print(f"  Error in scaling: {str(e)}")
            continue
        
        # === 1. FULL MODEL WITH ALL PREDICTORS ===
        if len(available_predictors) <= max_predictors:
            try:
                print(f"    Testing full model with {len(available_predictors)} predictors")
                
                # Fit logistic regression
                lr_full = LogisticRegression(random_state=42, max_iter=1000)
                lr_full.fit(X_scaled, y)
                
                # Cross-validation
                cv_scores_auc = cross_val_score(lr_full, X_scaled, y, cv=3, scoring='roc_auc')
                cv_scores_acc = cross_val_score(lr_full, X_scaled, y, cv=3, scoring='accuracy')
                
                # Feature importance
                coefficients = lr_full.coef_[0]
                odds_ratios = np.exp(coefficients)
                
                # Sort features by absolute coefficient
                feature_importance = list(zip(available_predictors, coefficients, odds_ratios))
                feature_importance.sort(key=lambda x: abs(x[1]), reverse=True)
                
                # Get performance metrics
                y_pred_proba = lr_full.predict_proba(X_scaled)[:, 1]
                auc_score = roc_auc_score(y, y_pred_proba)
                
                # Store result
                result_entry = {
                    'Analysis': analysis_name,
                    'Outcome': outcome,
                    'Model_Type': 'Full_Multiple_Logistic',
                    'Num_Predictors': len(available_predictors),
                    'Predictor_List': ', '.join(available_predictors),
                    'Sample_Size': len(subset),
                    'AUC_Score': round(auc_score, 4),
                    'CV_AUC_Mean': round(cv_scores_auc.mean(), 4),
                    'CV_AUC_Std': round(cv_scores_auc.std(), 4),
                    'CV_Accuracy_Mean': round(cv_scores_acc.mean(), 4),
                    'Model_Performance': 'Excellent' if auc_score > 0.8 else 'Good' if auc_score > 0.7 else 'Fair' if auc_score > 0.6 else 'Poor'
                }
                
                # Add top 3 predictors
                for i in range(min(3, len(feature_importance))):
                    pred_name, coef, odds_ratio = feature_importance[i]
                    result_entry[f'Top_Predictor_{i+1}'] = pred_name
                    result_entry[f'Top_Predictor_{i+1}_Coef'] = round(coef, 4)
                    result_entry[f'Top_Predictor_{i+1}_OR'] = round(odds_ratio, 4)
                
                # Fill empty slots if fewer than 3 predictors
                for i in range(len(feature_importance), 3):
                    result_entry[f'Top_Predictor_{i+1}'] = ''
                    result_entry[f'Top_Predictor_{i+1}_Coef'] = ''
                    result_entry[f'Top_Predictor_{i+1}_OR'] = ''
                
                multi_results.append(result_entry)
                print(f"    Full model AUC: {auc_score:.4f}")
                
            except Exception as e:
                print(f"    Error in full model: {str(e)}")
        
        # === 2. BEST 2-PREDICTOR COMBINATION ===
        if len(available_predictors) >= 2:
            try:
                print(f"    Testing 2-predictor combinations")
                
                best_auc = 0
                best_combo = None
                best_model = None
                
                # Test all 2-predictor combinations
                for combo in itertools.combinations(available_predictors, 2):
                    try:
                        combo_indices = [available_predictors.index(pred) for pred in combo]
                        X_combo = X_scaled[:, combo_indices]
                        
                        lr_combo = LogisticRegression(random_state=42, max_iter=1000)
                        lr_combo.fit(X_combo, y)
                        
                        y_pred_proba_combo = lr_combo.predict_proba(X_combo)[:, 1]
                        auc_combo = roc_auc_score(y, y_pred_proba_combo)
                        
                        if auc_combo > best_auc:
                            best_auc = auc_combo
                            best_combo = combo
                            best_model = lr_combo
                            
                    except Exception:
                        continue
                
                if best_combo and best_model:
                    # Cross-validation for best combo
                    combo_indices = [available_predictors.index(pred) for pred in best_combo]
                    X_best_combo = X_scaled[:, combo_indices]
                    
                    cv_scores_auc_combo = cross_val_score(best_model, X_best_combo, y, cv=3, scoring='roc_auc')
                    cv_scores_acc_combo = cross_val_score(best_model, X_best_combo, y, cv=3, scoring='accuracy')
                    
                    result_entry = {
                        'Analysis': analysis_name,
                        'Outcome': outcome,
                        'Model_Type': 'Best_2_Predictor_Combo',
                        'Num_Predictors': 2,
                        'Predictor_List': ', '.join(best_combo),
                        'Sample_Size': len(subset),
                        'AUC_Score': round(best_auc, 4),
                        'CV_AUC_Mean': round(cv_scores_auc_combo.mean(), 4),
                        'CV_AUC_Std': round(cv_scores_auc_combo.std(), 4),
                        'CV_Accuracy_Mean': round(cv_scores_acc_combo.mean(), 4),
                        'Model_Performance': 'Excellent' if best_auc > 0.8 else 'Good' if best_auc > 0.7 else 'Fair' if best_auc > 0.6 else 'Poor',
                        'Top_Predictor_1': best_combo[0],
                        'Top_Predictor_1_Coef': round(best_model.coef_[0][0], 4),
                        'Top_Predictor_1_OR': round(np.exp(best_model.coef_[0][0]), 4),
                        'Top_Predictor_2': best_combo[1],
                        'Top_Predictor_2_Coef': round(best_model.coef_[0][1], 4),
                        'Top_Predictor_2_OR': round(np.exp(best_model.coef_[0][1]), 4),
                        'Top_Predictor_3': '',
                        'Top_Predictor_3_Coef': '',
                        'Top_Predictor_3_OR': ''
                    }
                    
                    multi_results.append(result_entry)
                    print(f"    Best 2-predictor combo AUC: {best_auc:.4f} ({best_combo})")
                
            except Exception as e:
                print(f"    Error in 2-predictor analysis: {str(e)}")
        
        # === 3. BEST 3-PREDICTOR COMBINATION ===
        if len(available_predictors) >= 3:
            try:
                print(f"    Testing 3-predictor combinations")
                
                best_auc_3 = 0
                best_combo_3 = None
                best_model_3 = None
                
                # Limit combinations to avoid excessive computation
                all_3_combos = list(itertools.combinations(available_predictors, 3))
                if len(all_3_combos) > 20:
                    import random
                    random.seed(42)
                    test_combos = random.sample(all_3_combos, 20)
                else:
                    test_combos = all_3_combos
                
                for combo in test_combos:
                    try:
                        combo_indices = [available_predictors.index(pred) for pred in combo]
                        X_combo = X_scaled[:, combo_indices]
                        
                        lr_combo = LogisticRegression(random_state=42, max_iter=1000)
                        lr_combo.fit(X_combo, y)
                        
                        y_pred_proba_combo = lr_combo.predict_proba(X_combo)[:, 1]
                        auc_combo = roc_auc_score(y, y_pred_proba_combo)
                        
                        if auc_combo > best_auc_3:
                            best_auc_3 = auc_combo
                            best_combo_3 = combo
                            best_model_3 = lr_combo
                            
                    except Exception:
                        continue
                
                if best_combo_3 and best_model_3:
                    combo_indices = [available_predictors.index(pred) for pred in best_combo_3]
                    X_best_combo_3 = X_scaled[:, combo_indices]
                    
                    cv_scores_auc_combo3 = cross_val_score(best_model_3, X_best_combo_3, y, cv=3, scoring='roc_auc')
                    cv_scores_acc_combo3 = cross_val_score(best_model_3, X_best_combo_3, y, cv=3, scoring='accuracy')
                    
                    result_entry = {
                        'Analysis': analysis_name,
                        'Outcome': outcome,
                        'Model_Type': 'Best_3_Predictor_Combo',
                        'Num_Predictors': 3,
                        'Predictor_List': ', '.join(best_combo_3),
                        'Sample_Size': len(subset),
                        'AUC_Score': round(best_auc_3, 4),
                        'CV_AUC_Mean': round(cv_scores_auc_combo3.mean(), 4),
                        'CV_AUC_Std': round(cv_scores_auc_combo3.std(), 4),
                        'CV_Accuracy_Mean': round(cv_scores_acc_combo3.mean(), 4),
                        'Model_Performance': 'Excellent' if best_auc_3 > 0.8 else 'Good' if best_auc_3 > 0.7 else 'Fair' if best_auc_3 > 0.6 else 'Poor',
                        'Top_Predictor_1': best_combo_3[0],
                        'Top_Predictor_1_Coef': round(best_model_3.coef_[0][0], 4),
                        'Top_Predictor_1_OR': round(np.exp(best_model_3.coef_[0][0]), 4),
                        'Top_Predictor_2': best_combo_3[1],
                        'Top_Predictor_2_Coef': round(best_model_3.coef_[0][1], 4),
                        'Top_Predictor_2_OR': round(np.exp(best_model_3.coef_[0][1]), 4),
                        'Top_Predictor_3': best_combo_3[2],
                        'Top_Predictor_3_Coef': round(best_model_3.coef_[0][2], 4),
                        'Top_Predictor_3_OR': round(np.exp(best_model_3.coef_[0][2]), 4)
                    }
                    
                    multi_results.append(result_entry)
                    print(f"    Best 3-predictor combo AUC: {best_auc_3:.4f} ({best_combo_3})")
                
            except Exception as e:
                print(f"    Error in 3-predictor analysis: {str(e)}")
    
    # Create results DataFrame
    if multi_results:
        results_df = pd.DataFrame(multi_results)
        print(f"\n=== Analysis Complete ===")
        print(f"Generated {len(results_df)} model results")
        return results_df
    else:
        print(f"\n=== Analysis Complete ===")
        print("No valid models generated")
        # Return empty DataFrame with expected columns
        empty_df = pd.DataFrame(columns=[
            'Analysis', 'Outcome', 'Model_Type', 'Num_Predictors', 'Predictor_List', 'Sample_Size',
            'AUC_Score', 'CV_AUC_Mean', 'CV_AUC_Std', 'CV_Accuracy_Mean', 'Model_Performance',
            'Top_Predictor_1', 'Top_Predictor_1_Coef', 'Top_Predictor_1_OR',
            'Top_Predictor_2', 'Top_Predictor_2_Coef', 'Top_Predictor_2_OR',
            'Top_Predictor_3', 'Top_Predictor_3_Coef', 'Top_Predictor_3_OR'
        ])
        return empty_df

# Now run the rewritten analysis
print("=== RUNNING REWRITTEN MULTIPLE PREDICTOR ANALYSIS ===")

# Run multiple predictor analysis
multiple_pred_results = multiple_predictor_analysis(longitudinal_clean, long_predictors, long_outcomes, "Longitudinal_Multiple_Predictors")

# Check results
print(f"\nResults DataFrame shape: {multiple_pred_results.shape}")
print(f"Results DataFrame columns: {list(multiple_pred_results.columns)}")

# Filter and display results
if len(multiple_pred_results) > 0:
    
    # Filter for good performing models (AUC > 0.6)
    good_models = multiple_pred_results[multiple_pred_results['AUC_Score'] > 0.6].copy()
    good_models = good_models.sort_values('AUC_Score', ascending=False)
    
    print(f"\n=== MULTIPLE PREDICTOR ANALYSIS RESULTS ===")
    print(f"Total models tested: {len(multiple_pred_results)}")
    print(f"Good performing models (AUC > 0.6): {len(good_models)}")
    
    # Display all results first
    print(f"\n=== ALL MODEL RESULTS ===")
    all_display_cols = ['Outcome', 'Model_Type', 'Num_Predictors', 'AUC_Score', 'CV_AUC_Mean', 'Model_Performance', 'Sample_Size']
    print(multiple_pred_results[all_display_cols].round(3))
    
    if len(good_models) > 0:
        multi_display_cols = ['Outcome', 'Model_Type', 'Num_Predictors', 'AUC_Score', 'Model_Performance',
                             'Top_Predictor_1', 'Top_Predictor_1_OR', 'Top_Predictor_2', 'Top_Predictor_2_OR', 'Sample_Size']
        print(f"\n=== GOOD PERFORMING MODELS (AUC > 0.6) ===")
        print(good_models[multi_display_cols].round(3))
        
        # Summary of best models by outcome
        print(f"\n=== BEST MODEL PER OUTCOME ===")
        best_by_outcome = good_models.loc[good_models.groupby('Outcome')['AUC_Score'].idxmax()]
        if len(best_by_outcome) > 0:
            summary_cols = ['Outcome', 'Model_Type', 'AUC_Score', 'Predictor_List', 'Model_Performance']
            print(best_by_outcome[summary_cols].round(3))
    else:
        print("\nNo models achieved AUC > 0.6")
        print("Best performing models:")
        if len(multiple_pred_results) > 0:
            best_models = multiple_pred_results.nlargest(5, 'AUC_Score')
            print(best_models[all_display_cols].round(3))
    
    # Save results
    multiple_pred_results.to_csv('multiple_predictor_analysis.csv', index=False)
    if len(good_models) > 0:
        good_models.to_csv('good_multiple_predictor_models.csv', index=False)
    
    print(f"\nResults saved to CSV files")
    
else:
    print("No results generated - check data compatibility")

In [ ]:
# Cell: DIAGNOSTIC - Investigate Data Issues
def diagnose_longitudinal_data_issues(longitudinal_clean, long_predictors, long_outcomes):
    """
    Diagnose why we have insufficient data for analysis
    """
    
    print("=== LONGITUDINAL DATA DIAGNOSTIC ===")
    print(f"Longitudinal dataset shape: {longitudinal_clean.shape}")
    print(f"Predictors to analyze: {len(long_predictors)}")
    print(f"Outcomes to analyze: {len(long_outcomes)}")
    
    # Check if predictors and outcomes exist in the data
    print(f"\n=== COLUMN AVAILABILITY CHECK ===")
    missing_predictors = [p for p in long_predictors if p not in longitudinal_clean.columns]
    missing_outcomes = [o for o in long_outcomes if o not in longitudinal_clean.columns]
    
    print(f"Missing predictors: {missing_predictors}")
    print(f"Missing outcomes: {missing_outcomes}")
    
    # Check available columns
    available_predictors = [p for p in long_predictors if p in longitudinal_clean.columns]
    available_outcomes = [o for o in long_outcomes if o in longitudinal_clean.columns]
    
    print(f"Available predictors ({len(available_predictors)}): {available_predictors}")
    print(f"Available outcomes ({len(available_outcomes)}): {available_outcomes}")
    
    # Check missing value patterns
    print(f"\n=== MISSING VALUE ANALYSIS ===")
    
    if available_predictors:
        print("Predictor missing value counts:")
        for pred in available_predictors:
            missing_count = longitudinal_clean[pred].isna().sum()
            missing_pct = (missing_count / len(longitudinal_clean)) * 100
            print(f"  {pred}: {missing_count} missing ({missing_pct:.1f}%)")
    
    if available_outcomes:
        print("\nOutcome missing value counts:")
        for outcome in available_outcomes:
            missing_count = longitudinal_clean[outcome].isna().sum()
            missing_pct = (missing_count / len(longitudinal_clean)) * 100
            print(f"  {outcome}: {missing_count} missing ({missing_pct:.1f}%)")
    
    # Check complete cases for each outcome
    print(f"\n=== COMPLETE CASES ANALYSIS ===")
    
    for outcome in available_outcomes:
        print(f"\nAnalyzing complete cases for outcome: {outcome}")
        
        # Required columns for this outcome
        required_cols = available_predictors + [outcome]
        
        # Check data availability step by step
        print(f"  Total rows in dataset: {len(longitudinal_clean)}")
        
        # Check each column individually
        individual_counts = {}
        for col in required_cols:
            non_missing = longitudinal_clean[col].notna().sum()
            individual_counts[col] = non_missing
            print(f"  {col}: {non_missing} non-missing values")
        
        # Check complete cases
        complete_cases = longitudinal_clean[required_cols].dropna()
        print(f"  Complete cases (all columns): {len(complete_cases)}")
        
        if len(complete_cases) > 0:
            print(f"  Sample of complete cases:")
            print(complete_cases.head())
        else:
            print("  No complete cases found!")
            
            # Find which combinations have the most data
            print("  Checking pairwise combinations...")
            for pred in available_predictors[:5]:  # Check first 5 predictors
                pair_complete = longitudinal_clean[[pred, outcome]].dropna()
                print(f"    {pred} + {outcome}: {len(pair_complete)} complete cases")

# Run diagnostic
diagnose_longitudinal_data_issues(longitudinal_clean, long_predictors, long_outcomes)

# Let's also check the original longitudinal_data before preprocessing
print(f"\n=== ORIGINAL LONGITUDINAL DATA CHECK ===")
print(f"Original longitudinal_data shape: {longitudinal_data.shape}")
print(f"Original columns: {list(longitudinal_data.columns)}")

# Check if the preprocessing step removed too much data
print(f"\n=== PREPROCESSING IMPACT ===")
print("Original data missing values:")
for col in longitudinal_data.columns:
    if col in long_predictors + long_outcomes:
        missing_count = longitudinal_data[col].isna().sum()
        missing_pct = (missing_count / len(longitudinal_data)) * 100
        print(f"  {col}: {missing_count} missing ({missing_pct:.1f}%)")

# Check data types
print(f"\n=== DATA TYPES CHECK ===")
for col in longitudinal_data.columns:
    if col in long_predictors + long_outcomes:
        dtype = longitudinal_data[col].dtype
        unique_vals = longitudinal_data[col].dropna().unique()
        print(f"  {col}: {dtype}, unique values: {unique_vals}")

In [ ]:
# Cell: SIMPLIFIED ANALYSIS - Work with Available Data
def simplified_longitudinal_analysis(longitudinal_data, predictors, outcomes):
    """
    Simplified analysis that works with whatever data we have
    """
    
    print("=== SIMPLIFIED LONGITUDINAL ANALYSIS ===")
    
    # Use original data instead of preprocessed
    data = longitudinal_data.copy()
    
    results = []
    
    for outcome in outcomes:
        if outcome not in data.columns:
            continue
            
        print(f"\nAnalyzing outcome: {outcome}")
        
        for predictor in predictors:
            if predictor not in data.columns:
                continue
                
            # Simple pairwise analysis
            pair_data = data[[predictor, outcome]].dropna()
            
            if len(pair_data) < 5:
                continue
                
            print(f"  {predictor} -> {outcome}: {len(pair_data)} cases")
            
            # Check if both variables are binary
            pred_unique = set(pair_data[predictor].unique())
            out_unique = set(pair_data[outcome].unique())
            
            if pred_unique.issubset({0, 1}) and out_unique.issubset({0, 1}):
                # Binary analysis
                crosstab = pd.crosstab(pair_data[predictor], pair_data[outcome])
                print(f"    Crosstab:\n{crosstab}")
                
                # Chi-square test
                from scipy.stats import chi2_contingency
                if crosstab.shape == (2, 2):
                    chi2, p_val, dof, expected = chi2_contingency(crosstab)
                    
                    results.append({
                        'Predictor': predictor,
                        'Outcome': outcome,
                        'Sample_Size': len(pair_data),
                        'Chi2_Stat': round(chi2, 4),
                        'P_Value': round(p_val, 4),
                        'Significant': p_val < 0.05,
                        'Crosstab': crosstab.to_dict()
                    })
                    
                    print(f"    Chi2: {chi2:.4f}, p-value: {p_val:.4f}")
    
    if results:
        results_df = pd.DataFrame(results)
        significant_results = results_df[results_df['Significant'] == True]
        
        print(f"\n=== SIMPLIFIED RESULTS ===")
        print(f"Total tests: {len(results_df)}")
        print(f"Significant results: {len(significant_results)}")
        
        if len(significant_results) > 0:
            display_cols = ['Predictor', 'Outcome', 'Sample_Size', 'Chi2_Stat', 'P_Value']
            print("\nSignificant relationships:")
            print(significant_results[display_cols])
            
        return results_df
    else:
        print("No valid pairwise comparisons found")
        return pd.DataFrame()

# Run simplified analysis
simplified_results = simplified_longitudinal_analysis(longitudinal_data, long_predictors, long_outcomes)

In [ ]:
# Cell: CHECK DATA MERGE PROCESS
print("=== CHECKING DATA MERGE PROCESS ===")

# Let's examine the original wave data
print(f"Wave1 shape: {wave1.shape}")
print(f"Wave23 shape: {wave23.shape}")

# Check StFCID availability
print(f"\nStFCID in wave1: {'StFCID' in wave1.columns}")
print(f"StFCID in wave23: {'StFCID' in wave23.columns}")

if 'StFCID' in wave1.columns and 'StFCID' in wave23.columns:
    wave1_ids = set(wave1['StFCID'].dropna())
    wave23_ids = set(wave23['StFCID'].dropna())
    
    print(f"Unique IDs in wave1: {len(wave1_ids)}")
    print(f"Unique IDs in wave23: {len(wave23_ids)}")
    print(f"Overlapping IDs: {len(wave1_ids.intersection(wave23_ids))}")
    
    # Check if the merge is working
    if len(wave1_ids.intersection(wave23_ids)) == 0:
        print("WARNING: No overlapping IDs - merge will fail!")
        print("Sample wave1 IDs:", list(wave1_ids)[:5])
        print("Sample wave23 IDs:", list(wave23_ids)[:5])

# Check predictor and outcome availability in original data
print(f"\n=== ORIGINAL DATA COLUMN CHECK ===")
print("PREDICTORS availability:")
for pred in PREDICTORS:
    in_wave1 = pred in wave1.columns
    in_wave23 = pred in wave23.columns
    print(f"  {pred}: Wave1={in_wave1}, Wave23={in_wave23}")

print("\nOUTCOMES availability:")
for outcome in OUTCOMES:
    # Check for outcome with suffixes
    w23_col = outcome + '_w23'
    
    in_wave23_w23 = w23_col in wave23.columns
    
    print(f"  {outcome}: w23={in_wave23_w23}")

# Try a manual merge to see what's happening
print(f"\n=== MANUAL MERGE TEST ===")
test_merge = pd.merge(wave1[['StFCID']], wave23[['StFCID']], on='StFCID', how='inner')
print(f"Test merge result: {len(test_merge)} rows")